## Install Dependencies and Import Modules

In [1]:
!pip install openai
!pip install scikit-learn
!pip install nltk
!pip install umap-learn
!pip install tqdm
!pip install numpy

In [2]:
import openai
import nltk
import umap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/lior/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/lior/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lior/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Configuration

In [ ]:
Configure your OpenAI API key here

In [3]:
# Configure your key here
OPENAI_API_KEY = 'YOUR_KEY'

OPENAI_CHAT_COMPLETIONS_MODEL = 'gpt-4'

OPENAI_EMBEDDING_MODEL = 'text-embedding-3-large'

N_CLUSTERS = 3

DATA_PATH = 'ML_Screener.xlsx'

## Load and Preprocess the Data

In [4]:
# # Load the Excel file
# df = pd.read_excel(DATA_PATH)

# client = openai.OpenAI(api_key = OPENAI_API_KEY)

# def openai_process_text(text, prompt):
#     # Use OpenAI's GPT model to process text
#     response = client.chat.completions.create(
#         model=OPENAI_CHAT_COMPLETIONS_MODEL,
#         messages=[{'role': 'user', 'content': '{}: {}'.format(prompt, text)}],
#         max_tokens=100
#     )
#     return response.choices[0].message.content

# def openai_translate_and_remove_emojis(text):
#     return openai_process_text(text, "Translate to english, remove emojis")

# # Filter words that carry no information, AKA noise
# filtered_words = ['#xinjiang', '#fy', '#fyp', '#foryou', '#foryourpage', '#viral', '#trending', '#trendingtiktok']
# def filter_words(text):
#     return ' '.join([word for word in text.split() if word not in filtered_words])

# def remove_punctuation(text):
#     # Tokenize text
#     tokens = word_tokenize(text)
#     # Remove punctuation from tokens
#     words = [word for word in tokens if word not in string.punctuation]
#     # Rejoin words into a string
#     return ' '.join(words)

# def remove_stopwords(text):
#     # Tokenize text
#     tokens = word_tokenize(text)
#     # Remove stopwords
#     words = [word for word in words if word.lower() not in stop_words]
#     # Rejoin words into a string
#     return ' '.join(words)

# tqdm.pandas(desc="Openai preprocessing")
# df['texts'] = df['texts'].progress_apply(openai_translate_and_remove_emojis)
# df['texts'] = df['texts'].str.lower()
# df['texts'] = df['texts'].apply(filter_words)
# #df['texts'] = df['texts'].apply(remove_punctuation)
# #df['texts'] = df['texts'].apply(remove_stopwords)

#### Run this premade version instead to save on API calls:

In [5]:
preprocessed_texts = ['Winter trip #tancuong #chinatravel #xinjiang #dptmino', 'Truth about Xinjiang Uyghurs #xinjiang #uyghurmuslims #uyghurs #islam #ramadan #china #socialism #communism #marxism | #turkey #eastturkestan #uyghurlivesmatter #uyghurgenocide', 'Turkish with Uighurs #travel #travels #trips #uyghur #china #xinjiang #easternturkestan #fy #fyp', 'If you truly want to immerse yourself in the vibrant culture of Xinjiang, go directly to Kashgar! This ancient city is like a time capsule, with its streets filled with native residents. Explore hidden corners and alleys, and get ready to be pleasantly surprised at every turn. #Xinjiang #culture', 'White, symbolizing purity and sacred light, cleanses our minds and brings us peace. #Xinjiang #colors #white #colorfulXinjiang #snow #ice #clouds #scenery #nature', 'What is the main dish of the people in Xinjiang? #travel #food #trends #xinjiang', 'Life of Uighur Muslims in Xinjiang, mainland China #chinesemuslim #muslimuighur #chinamuslim #china #trendingvideo #trendingtiktok #fyp #chinatiktok #muslimhui #ramadan2023 #ramadan #ramadanmubarak #ramadanmubarak2023 #ramadhanmubarak2023 #ramadan2023inshallah', 'How does it feel when all the expectations for spring are encapsulated in a thin pancake? #visitxinjiang #xinjiangisaniceplace #thisisxinjiang #xinjiangstyle #xinjiangportrait #explorexinjiang #travelinxinjiang #xinjianglife #china #xintaste #xinjiangdelicacy #xinjiangspeciality #chinesefood', 'We, as a family, had a great time with all sorts of fascinating activities in Xinjiang! (Xinjiang Part 2)', 'Credibility of BBC? China spokesperson Huachunying speaks on media about Xinjiang. For your page, for you.', "Video from inside one of Xinjiang Concentration Camps shows Uyghur men are handcuffed and are forced to sing 'Without the Communist Party, there would be no new China.'", '#Uyghur #Uzbekistan #Xinjiang #Turkey #Turkic #Kazakhstan #Kyrgyzstan #Tatarstan #Azerbaijan #Turan #Turk', "This is our amazing travel guide for Northern Xinjiang! Reach out to us directly, and we'll assist you in arranging everything! #Xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", 'Natural Kashgar dance, China, travel, Xinjiang China, for your page, Uyghur Muslims, Xinjiang Uyghurs, Uyghur, Xinjiang, fitness motivation.', 'The inhabitants of the ancient city of Kashgar display their unique musical instruments and costumes, and their purest smiles, to give the world a glimpse into a unique blend of Xinjiang and Chinese culture. #Xinjiang #ancientcity #music', 'Muztagh Ata is an emblem of the Pamirs in China’s Xinjiang Uygur Autonomous Region. By viewing aerial photographs, we can catch a glimpse of the stunning landscape, which consists of both glaciers and snow-covered mountains. #Xinjiang #scenery', 'Some beautiful girls from Xinjiang. #interview #beautifulwomen #beauty #charmingwoman #foryoupage', 'Xinjiang Chinese Muslim dishes at @Anas Restaurant, Halal Toyyiban Approved. Address: suite apartment, Danau Kota, Jalan 1a/6, Taman Setapak Indah, 53300 Kuala Lumpur. #chinesemuslimfood #halalfoodie #muslimfoodie #muslimchinesefood #roastlamb #ridzuanongpodcast #halalfood', 'On the 15th, in Turpan, Xinjiang, the hair of tourists stood on end in the air. Netizens: Hurry up and leave, the static electricity is too strong, there will be lightning soon, pay attention to safety.', 'Xinjiang is wonderful, and its people are happy. #xinjiang #xinjiangchina #uyghur #uyghurmuslims #islam #uyghurlivesmatter #urumqi #eastturkestan #turkey #china #socialism #communism', 'With joyful music playing, the Tajiks danced to warmly welcome the guests! #xinjiang #Tajik', "What an exhilarating scene of a plentiful harvest! In Xinjiang, China's largest high-quality cotton production base, over 6,900 cotton-picking machines create a roar as they move through the vast cotton fields. They form an impressive spectacle and finish the picking task efficiently! Xinjiang has a variety of cotton species, such as Fine Staple cotton, Upland Cotton, and Long-staple Cotton. It also naturally breeds Colored Cotton. Altogether, the cotton output from Xinjiang makes up approximately 20% of the worldwide cotton production. Xinjiang's cotton industry has developed into an efficient, energy-saving, and highly valued integrated chain, spanning from cotton planting, processing and trading, to dyeing, spinning, and weaving. The cotton industry in China is growing, flourishing and evolving in Xinjiang, advancing towards a brighter future! #insiders #XJonair #diveintoXJ #cotton #machine #industry", 'Xinjiang China, Xinjiang is a place where many Muslim people live #china #xinjiangchina #fyp #inchina #fyp.', 'Costs to travel to Xinjiang #travel #xinjiang #kieutravel #fyg #NANNO #TVSHOWHAY #china #tourism #Cartooncharacter', 'How people celebrate and welcome New Year 2023 in Xinjiang with a large bonfire #xinjiang #xinjiangchina #hello2023 #china #newyear #happynewyear', 'Aerial View of Korla City, Xinjiang, Korla', 'Roads covered with berries in the Taklimakan Desert, Xinjiang, China. #china #construction #chinaconstruction #road #desert #desertroad', 'Beautiful Uyghur female police officer at a market in Urumqi, China #foryourpage #Uyghur #UyghurTikTok #UyghurMuslims #Police #PoliceOfficer #PoliceTikTok #China #XinjiangChina #XinjiangTravel', 'The hearty breakfast in Xinjiang, China', "Uyghurs are praying in mosques in Xinjiang, China - This is the real Xinjiang. I strongly advise the West to stop using pictures of non-Uyghur Asians from other countries and misrepresenting them as 'Uyghurs'. The fact that Western media labels any Asian wearing a Hijab as a Uyghur is ethically dishonest. However, they are fully aware of what they are doing, as we saw two decades ago when Western media spread falsehoods about Afghanistan and Iraq.", 'The windiest road in XinJiang, China, 30 kilometers with 639 turns. Dare to try? #fyp #sharpturns', '7.1 magnitude earthquake strikes Xinjiang.', 'Muslim Lives Matter (only in China?) #uighur #uyghurmuslims #xinjiang #praying #imperialism #cia #communist #marxist #china', 'Costs for going to Xinjiang #travel #xinjiang #kieutravel #fyg #NANNO #TVSHOWHAY #china #tourism #CartoonCharacter', "In October, the poplar forests of Kashgar resemble jumping flames. If you ever visit Xinjiang, don't forget to check out and appreciate the captivating landscape of the poplar forests! #Xinjiang #autumn #forest", 'Xinjiang, China #beautifulscenery #tiktok #foryourpage #beautiful #china', 'This is not a dinosaur! The "Ace of Xinjiang Barbecue" ought to be the Naan Pit-Roasted Camel. In Wobuxun Village, Tacheng, Xinjiang, the barbecue chefs cooked a 1,040-kilogram camel. The barbecue chef from Bachu County, Xinjiang, stated that they used 32 ingredients for this camel roast. It required more than 30 hours of work by seven staff members to roast it completely. The camel meat is fragrant, savory, and slightly salty, and its texture is even softer than beef. Tourists are full of compliments for the roasted camel meat! Photo credit: douyin: jiangjiang.', 'The dances of ethnic minorities in Xinjiang are unique!', '#xinjiang #china #uighurmuslims #novibasuki" is already in English. They are hashtags related to the Xinjiang region in China, the Uighur Muslims who live there, and potentially someone or something by the name of Novi Basuki. Without more context, it\'s hard to give an exact translation.', 'When traveling to Xinjiang, you must know this #traveltochina #china #wanderingjack #xinjiangfood #TikTokEntertainment #chinatravel #urumqi #tourchina', 'Muslin in Xinjiang, China', 'Xu Ke / Feng Ming with his daughter vlog, like father like daughter, so cute and pretty  #xuke #xuke (in Chinese) #fengming #kaixinjiang #xumuyan #xumuyan (in Chinese) #happyandbubblytalk (in Chinese) #foryourpage #thelastimmortal #shenyin #foryou', 'Xinjiang has a unique tradition that sets it apart #xinjiang #kashgar #costume #tradition #china', 'Beautiful Xinjiang', "Here's our 10-day Xinjiang schedule! If you like it, let's plan a trip together next year! Feel free to reach out to us, and we can arrange the Xinjiang experience you want! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", 'Xinjiang, with its extensive area, is one of the wealthiest regions in China when it comes to wind energy resources. Watching the wind turbines spin beneath the golden sunset, it feels as though time moves a little slower. #xinjiang', '#Xinjiang #China #Uyghur #FridayPrayer #NoviBasuki', '7.1 magnitude earthquake strikes Xinjiang. A powerful 7.1 magnitude earthquake hit Xinjiang.', "#Gweilo60 on the Real Victims of #Xinjiang #Xinjiang Terrorism's Real Victims", 'Step into the Narat Grassland in Xinjiang! Experience pure joy and endless wonder with every step you take. #travel #tour #China #Xinjiang #Narat #!China #acrossChina #wonderfulChina #amazingChina #WowChina #like #follow #fyp', 'On the 15th, in Turpan, Xinjiang, the hair of tourists stood on end in the air. Netizens: Hurry up, the static electricity is too strong, there will be lightning soon, pay attention to safety.', 'Escape to the peacefulness in Kanas, Xinjiang - a village where time seems to freeze. Surround yourself with calmness, the embrace of nature, and the comforting patterns of local life. #Kanas #VillageLife #Nature #Xinjiang #Journey #fyp #quiet #travel', "Do not trust the western media. If you don't believe me, take a look at the Xinjiang videos on TikTok. MOST THINGS ARE NOT TRUE! #uyghur #uyghurgenocideisnthappening #china #mylord #idontcarehowlong #historyedit @Tamer", 'The most beautiful scenery in Xinjiang #scenery #nature #spring #beautiful #beauty #foryou #fyp #f', '#Chinese #MiddleEastern #food #Uyghur #Xinjiang', 'Embark on a journey into the mystical beauty of Altay, Xinjiang, and immerse yourself in the moment as you watch one of the most stunning sunsets unfold right before your eyes. #ForYourPage #Nature #Winter', 'This is in Urumqi, Xinjiang, China #urumqi #xinjiang #xinjiangchina #china #uyghurmuslims #islam #turkey #uyghurlivesmatter #eastturkestan #socialism #communism', 'Replying to @hartantiyunida95 #dewidilraba #dilrabadilmurat #asianbeautytiktok #2023 #queen #goddess #beautiful #xinjiang #uighur #fyp #foryou #video #viral #tiktok', 'Xinjiang has been off-limits to foreign reporters for several years, but we had the opportunity to visit Kashgar in southern Xinjiang as part of a strictly supervised media tour arranged by the Chinese government. By David Lipson, ABC International Team #Xinjiang #China #Tourism #ABCNews', 'Experience a unique concert of the Kyrgyz people on the Pamir Plateau in Xinjiang! #xinjiang #Kyrgyz', 'Here is the beautiful healing landscape of Xinjiang, China.', 'The most winding road in XinJiang, China, has 639 turns in 30 kilometers. Dare to try? #fyp #sharpturns', 'Nightlife atmosphere in Urumqi, Xinjiang Uygur Autonomous Region, China #urumchi #urumqinightlife #xinjiangnightlife #xinjiang #freeuyghurs #freeuyghurs #freeuyghurs #chinatiktok #china #chinesetiktok #nightlifeinurumchi #urumchi #nightlifeinxinjiang #dunganpeople #UighursinChina #FreeUighurs #uygurs #freedomforuygurs #UighurturmoilinChina #urumchi #urumchinights #urumchi #Urumqi #fy #fyp #fyp #fypviral', "Let's see how thick the snow is in Xinjiang~ #snow #heavysnow #tiktok #like #fyp #xinjiangchina #xinjiang", 'Residing in China has unveiled the reality about Xinjiang and exposed the Western propaganda concerning the Uyghurs. #China #Xinjiang', "Yili - Xinjiang ; Uighur (شىنجاڭ)  (伊犁Yīlì ) - (新疆 xīnjiāng ) China. Yili is an autonomous prefecture for the Kazakh people in Northern Xinjiang, China, one of five autonomous prefectures in Xinjiang. The city of Yining is its capital. It borders Mongolia, the Russian Federation, and the Republic of Kazakhstan from the northeast to the southwest, with a border line of 2,019 kilometers. Including Khorgas, Bakhty, and Jeminay, there are 9 national level ports of entry. With its unique location advantage, Yili has become an important commercial center and also a gateway to the west. Zhaosu County, Ili Kazakh Autonomous Prefecture, Xinjiang is famous as the 'Hometown of the Sky Horses of China’. The horses here are strong and large, able to run for a long time and very fast. Every summer, local herdsmen will drive herds of horses to the River to drink and bathe.  #travel #yili #Xinjiang #Uighur #شىنجاڭ #China", 'Good night, Urumqi. #fyp #Xinjiang #Urumqi #urumqi_xinjiang #sunset #goodnight #city', 'The breathtaking scenery in Xinjiang for part 3 @Breathtaking scenery. #travel #scenery #adventure #nature #China #Xinjiang #foryourpage #fantasy', 'Useful itinerary for those preparing to go to Xinjiang. #tancuong #xinjiang #chinatravel #dptmino', "Cry of the Uyghurs! For so many years, the Chinese have ignored the cries of the Uyghurs who are enduring genocide and concentration camps. It's time to wake up, China! #Uyghurs #ForFreedom #concentrationcamp #XinJiang #Genocide #ChinaUprising #A4Revolution #ChinaProtest #DownwithXijinping #DownwithCCP #Xijinping #ForFreedom", "99% of people don't know about the dangers in Xinjiang, China #tiktoktravel #traveltiktok #jackwandering #chinatravel #xinjiang #chinatravel  #turfan", "Does the Thien Son Snow Lotus actually exist? Or is it just a fictional flower in the martial arts stories (or movies) that we've heard of? The Thien Son Snow Lotus is a rare medicinal herb that grows on the snow mountains in Xinjiang, at an altitude of 5000m above sea level. It only blooms once every 7-8 years, and the blooming period only lasts a month before it dies. The rarity of this flower has led the Chinese government to protect it, and afterwards, it was artificially planted on Tian Chi Lake in the Tian Shan mountains. #dudibuitravel #dulichtrungquoc #traveltochina #chinatrip #tianshanxuelian #xinjiang #tianshan", "A new highway is under construction in China's largest desert. #foryourpage #China #chinesetiktok #Xinjiang #techtok", 'Dive into this mesmerizing golden world, where fall in Kanas, Xinjiang outshines everything else. The awe-inspiring beauty is worth the long journey. #xinjiang #scenery #autumn.', 'The \'genocide\' accusation in Xinjiang is nothing more than an outrageous lie spread by the U.S." #China #Xinjiang #US #America #realxinjiang #xinjiangchina #uyghur', '#Uighur #Xinjiang', 'Islamic figures around the world defend Xinjiang. #islam #islamic #arabtiktok #arabs #asia #muslim #fyp', 'Happy to see Tacko putting in effort #tackofall #xinjiang #xinjiangchina #cba #overseasbasketball #tacko', 'The most distant place from the sea in the world is in China! Everyone should visit at least once! #xinjiang #travellife #traveltochina #mustvisitplaces #viral #fyp', "Are Uyghurs being forced into labor in Xinjiang? Let's ask them! #uyghur #sinophobia #china #xinjiang #chineseculture #CapCut", 'Xinjiang Concentration Camps China Xinjiang', 'Uyghur language class #china #Xinjiang #uyghur #xinjianguyghurs #xinjiangtravel #xinjiangchina ... #UyghurShortCourse', 'Chinese Uyghur Muslim dress in Xinjiang #ForYourPage #Uyghur #UyghurTiktok #UyghurMuslims #ChineseMuslims', 'Tibetan boys #Tibetan #nomad #cowboy #lifestyle #foryoupage #centralasia #uyghur #xinjiang #freetibet #tibet', "An expatriate from Israel visited cotton farms in Xinjiang. Thank you for exploring Xinjiang and interacting with local farmers. It's time to book a ticket to Xinjiang #uyghur.", 'Do Muslim lives only matter in China? #uighur #uyghurmuslims #xinjiang #praying #imperialism #cia #communist #marxist #china', 'Location: Turpan, Xinjiang Autonomous Region, China. #turpan #xinjiang #xinjiangtravel #uyghurtour #uyghurtraditional #uyghurtraditionalclothing #xinjiangtourism #chinatravel #travelchina #chinatourism #china', 'This girl is from Xinjiang and she is Uyghur. Do you want to visit Xinjiang? #xinjiang #xinjiangchina #xinjianguyghurs #uyghur #uyghurs #uyghurpeople #uyghurgirl #china', "Replying to @Delilah_Lina Love and justice Genocide of Uighur Muslims is taking place in Xinjiang, yet the Islamic world has chosen China's money over China's Muslims. #sheistheone16 #silenceisviolence #uighur #saveuighurmuslims #corrupt #fyp #gravitasplus #wion #thumbsup #fyp #fy", 'The present of the Uyghur people is our future! No concentration camps, Free Uyghurs!! Dissolve the concentration camps, return freedom to the Uyghur people!! #XinjiangConcentrationCamp #Uyghur #WhitePaperRevolution #A4Revolution #Uyghurs #Uyghur #Xinjiang #concentrationcamp #Genocide #a4revolution #chinauprising #chinaprotest', "Don't go overseas, because visiting Xinjiang is like experiencing the entire world in one location! #xinjiang #travellife #traveltochina #mustvisitplaces #viralvideo #fyp", 'The breathtaking scenery in Xinjiang for part 6 @Breathtaking scenery. #travel #scenery #adventure #nature #China #Xinjiang #foryourpage #fantasy', 'Do you want to step into this painting? #Grassland #Xinjiang #Tekes #Nature #Mountain #Scenery', 'Too much into role playing. Want more virtual tours? #centralasia #travelguide #uyghur #xinjiang #bazaar #kanaslake #fyp #xyzbca #cultures #travel', 'So is there any problem with Xinjiang cotton? #China #Chinese #Mandarin #Language #Study #Learn #ChineseLanguage #Chinese #Cotton #Xinjiang #Uyghur #Farm #Bilingual #Vicali', 'Beautiful Uyghur women from Xinjiang, China #XinjiangChina #Uyghur #Muslim #Muslimah #Turkmenistan #XinjiangUyghur #365DaysQuarantine #foryou #fy', 'Why does the US want to separate Xinjiang from China? #us #china #islam #muslim #asia #middleeast #geopolitics #fyp', 'Beautiful Uyghur girl, welcome to Xinjiang. #china #xinjiang #girls #model', 'Driving on the Pamir Plateau in Xinjiang, the magnificent snow-capped Kunlun Mountains are gradually revealed. What a breathtaking view! #xinjiang #kunlun', 'The ending is so touching #TikTokSoundFeature #ChineseMuslim #Muslim #China #HalalFood #XinjiangFood #Culinary #Viral #ViralVideo #ViralTikTok #FYP #FYPシ #FYPViral #FYPPage #FYP [Repeated] #FYPシ #FYPSounds', 'Saw Kim Kardashian in Xinjiang! Would you be interested to take a look? #xinjiang #traveltochina #bigbottoms #traveltiktok #travellife #viralfood #roastedlamb #viralvideo #fyp', 'Night Market in Xinjiang at 11 pm #Xinjiang #China #XinjiangTravel #XinjiangUyghurs #XinjiangUyghur #Uyghur', 'The Tajiks of Xinjiang, also known as Mountain Tajiks, are a group of Pamiris who reside in the Pamir mountains within the Taxkorgan Tajik Autonomous County, situated in Xinjiang, China. #greenscreen #tajiks #xinjiang', 'EP1 | Visiting Xinjiang, China for the first time. Experience the mysterious culture that is different and unique. Enjoy the beauty of the desert and breathe in the fresh air from the Tianshan Mountains. If you have never visited it, you must come here #mikaXxinjiang #Visit #Experience #China #Xinjiang #Journey #Adventure #Exploration #Travel #Culture"\nNote: The translation may not be perfect as the hashtags are broad and can refer to a variety of things.', '7.1 earthquake does not interrupt surgeons during operation! Security cameras show the exact moment the 7.1 earthquake rocked Xinjiang, China, but doctors continued with the life-or-death craniotomy while everything around them moved. #earthquake #china #nature #captured', "Celebrating the 73rd anniversary of the People's Republic of China! #xinjiang #china #nationalday", 'Children from Xinjiang bring sheer delight to the Forbidden City with their captivating dance! #foryoupage #viral #trending #dance #Chinese #tiktok', 'Street interview in Urumqi, Xinjiang, China #streetinterview #china #xinjiang #xinjiangchina #xinjianguyghurs #uyghur #gaychina #gay', 'The untruths spread by the West about Xinjiang are gradually being uncovered. #west #us #humanrights #propaganda #germany #german #volkswagen #fyp', 'UN: China is violating Uyghur human rights. #learnontiktok #china #worldnews #humanrights #muslim #xinjiang #uyghur #prison #un #unitednations', 'Palestine reaches out to the US and UN as Israeli forces attack worshippers again at Al-Aqsa mosque. #xinjiang #china #muslim #ramadan #israel #palestine', 'At 10 pm in Xinjiang, people start going to the market to cook rice #travel #xinjiang #trend #RunawayPrincess #GreatTVShow', 'Xinjiang cuisine, so delicious! #Tiktok #rurallife #rural #nature #countrylife #countryliving #farmlife #ruralphotography. Do you enjoy the comfortable rural life? Please follow me, thank you.', 'Fall in love with the captivating Baisha Lake in October! The blue sky reflected on its peaceful waters will lift your spirits all month long. #Xinjiang #travel #scenery', '#Xinjiang #China #SaveXinjiang #ForYourPage', 'This is the Ken Swart hydraulic engineering project in Xinjiang. "Ken Swart" is a term in Mongolian that refers to a place where waters converge. It is also recognized as the longest and most magnificent Danxia landform in China, famously referred to as the "Geographical Landscape of the Tianshan Mountains". #xinjiang', '10 Days in Xinjiang: A Journey Beyond Imagination! From vibrant Urumqi to the enchanting Kanas, join us for an adventure like no other. #xinjiang  #travellife  #traveltochina  #mustvisitplaces  #traveltip  #viral  #fyp', 'What is #Xinjiang really like? I traveled to #Urumqi to find out for myself. #joinme #China #Shanghai', 'Han & Uyghur wedding in Karamay, Xinjiang #chinatiktok #uyghurmuslims #Uyghur #xinjianguyghurs #xinjiang', 'The substantial breakfast in Xinjiang, China', 'The first snowfall of 2024 in Kashgar city in Xinjiang region of China #Kashgar #Xinjiang #China #snow #video #vlog #tiktok #discover', 'The only city in China without traffic lights #scenery #xinjiang #bagua #city #incredible #foryou', 'Explore the winter spectacle at Bohu county, Xinjiang where the beautiful Bosten Lake is located. This hidden gem captivates visitors with its breathtaking landscapes. #ForYourPage #XinjiangSnow #nature #winter', 'Unable to choose between Southern or Northern Xinjiang? Why not explore both! Check out our 14-day program that covers both regions of Xinjiang! #xinjiang #china #traveltips #travellife #traveltochina #mustvisit #vacation #scenery #fyp #viral', 'Experience a unique concert of the Kirgiz people on the Pamir Plateau in Xinjiang! #xinjiang #Kirgiz', 'In the Xinjiang Uyghur Autonomous Region #china #travelvlog #xinjiang #xinjiangfood #uyghur #chinavlog #chinatravel #China #Chinatrip #Chinatravel #Chinese food #cuisines #tohokudaigaku #unemployment #NHK #unemployedtrip #foodvlog #Xinjiang #Xinjiangcuisine #Uyghurnation #japanesevlog #travelvlog', 'Western media lies about China, Xinjiang, Uyghur truth. Chinese, Western media, rumor, propaganda, global news.', 'Ice push, travel, China, Sayram Lake, Xinjiang, ice push.', 'Do you have any friends who are fond of Jade from Xinjiang? Come follow me to discover more incredible gemstones. #gem #gems #gemhunting #agate', 'Just laughing and wanting to collapse #herbstore #XinjiangBeauty #HighSchoolGirls #AttractiveStudents #XinjiangStudents #BeautifulGirls #XinjiangHighSchool #StudentBeauty #XinjiangGirls #AttractiveHighSchoolers #XinjiangCharm #HighSchoolBeauty #XinjiangWomen #StudentCharm #XinjiangTeens #AttractiveTeens #XinjiangYouth #HighSchoolCharm #XinjiangLadies #StudentGrace #XinjiangFemales #XinjiangBeauty #MiddleSchoolStudents #AttractiveStudents #XinjiangStudents #BeautifulGirls #XinjiangMiddleSchool #StudentBeauty #XinjiangGirls #AttractiveMiddleSchoolStudents #XinjiangAttraction #HighSchoolBeauty #XinjiangWomen #StudentAttraction #XinjiangYouth #AttractiveYouth #XinjiangYouth', "Follow PaTh to taste the food of Xinjiang Uygur Autonomous Region of China along the 'Belt and Road'! Use the same filter and join our 'flowers along the way' journey! #beltandroad #xinjiang #xinjiangchina #xinjiangtravel #foodie #delicious #PaTh #OneBeltOneRoad #Xinjiang #Cuisine", 'Dancing! #Xinjiang #Uyghur #Uygar #Muslim #Muslimah #China #365DaysQuarantine #XinjiangChina #XinjiangUyghur #ForYouPage #ForYou', 'Interview a cute girl from Xinjiang #interview #xinjiang #havu2904" is already in English.', 'Do you find the little ones of Ngo Duy Nhi cute? #travel #foryou #xinjiang #ughurmuslims', "This is our mind-blowing travel guide for Northern Xinjiang! Reach out to us directly, and we'll assist you in arranging everything! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", 'Grandpa and two beauties dance the traditional Xinjiang dance in the ancient city of Kashgar, Xinjiang. #Xinjiang #XinjiangChina #XinjiangUyghurs #Uyghur #UyghurMuslims #China', 'Interview with bestie from Xinjiang #beauty #beautiful #cute #lovely #beauty #beautifulwoman #charming #Asian', 'The beauty of Xinjiang is simply captivating! When are you planning your trip? #Xinjiang #TravelGoals #travel #fyp #nature #mountains #forest', 'What activities are there to do in Xinjiang during the autumn season? #learnUyghur #china #Xinjiang #uyghur #xinjianguyghur #foryoupage #foryou #xinjianguyghurs #EidAlAdha', 'I love Xinjiang #xinjiang #urumqi #kashgar #uyghur #uyghurmuslims #chinesemuslim #china #communism #islam #socialism #muslim', 'Major historical and cultural monuments under state protection" are not being abolished! #uyghur #xinjiang #muslimah #xinjiangchina #china #fyp #uygur', 'Lady from Wulumuqi City, Xinjiang Uyghur Autonomous Region, China. #chinesegirl #chinesewoman #chineselady', '#KFC #Xinjiang #China #BBC #Uighur #UK #vlogger #fyp', '#Xinjiang #Urumqi #Ili', 'Girls dancing at Uyghur weddings in Xinjiang, China, are freer and more secular than Muslims in the vast majority of the world!"#uyghursmuslims #uyghursazmakani #saveuyghurs #traineesofXinjiangCamps #XinjiangCamps #rescueuyghurs #Uyghurhumanrights #wanttohelpUyghurs #donotallowUyghurbombing #wanttosaveUyghurs #Uyghurs #FreeUyghurs #xinjiang #Xinjiang #Uighurs #wearecapturingUighursTikTok #Uighurs #Uighursbeauties #UighursinChina #UighursinTurkey #freedomforUighurs #Uighursarenotalone #EastTurkistan #EastTurkistan #China #China #Uyghurs #Uyghurs #China #China #foryou #foryoupage #viral #fypage.\nThe untranslated parts are hashtags which include terms for Uyghurs, Xinjiang concentration camps, Uyghur human rights, wanting to help Uyghurs, China, Xinjiang and the For You Page (fyp).', 'Traditional, characteristic wedding in Xinjiang Uyghur, China. #Xinjiang #China #Uyghur #XinjiangUyghurs #for your page #popular trend', "The dance and music traditions of Xinjiang are a lively representation of the region's multiculturalism, influenced by various ethnic groups such as the Uygur, Kazak, Kyrgiz, and Tajik. The distinctive mix of traditional music and dance is a fascinating part of Xinjiang's abundant cultural heritage. Thank you for sharing your beautiful culture with the world, #Xinjiang! #Uygur #Kazak #Kyrgiz #Tajik #traditionalmusic #dancing #culturalheritage #foryourpage #Yining", 'Took a journey to Xinjiang and found the purpose of life. Our group tour to Xinjiang is now open! Come and ask us now! #xinjiang #travellife #traveltochina #mustvisitplaces #spring #springscentsation #viral #fyp', "Did you know that Xinjiang has both Southern and Northern regions? Watch this video for immediate understanding! Let's discover it together! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltips #viral #fyp", "Muztagh Ata is a symbol of the Pamirs in China's Xinjiang Uygur Autonomous Region. Through aerial photos, we can catch a glimpse of the stunning landscape that includes both glaciers and snow-covered mountains. #Xinjiang #scenery", 'Do you like the bazaar in Xinjiang? #Xinjiang #xinjiangchina #uyghur #Uyghur #cuisine #cuisinetiktok #China', 'Railway in the Taklimakan desert in Xinjiang. #China #Taklimakan #TaklimakanDesert #Railway', 'Part 1: Top 10 Treasures of Xinjiang - A Must-See for Revealing the Beauty of Xinjiang #xinjiang #travellife #traveltochina #mustseeplaces #viral #fyp', '#Bagua City, #Xinjiang #BaguaCityXinjiang', 'Daily life in Kashgar, Xinjiang #kashgar #xinjiang #uyghur #china" is already in English. It\'s a social media post likely related to everyday life experiences in Kashgar, a city in the Xinjiang region of China, which has a significant Uyghur population. The hashtags are used to categorize the topic or theme of the post.', 'Uyghur girls performed an impromptu dance in the ancient city of Kashgar. A Uyghur girl improvised a Kazakh dance, moving with confidence in front of the camera and exhibiting a uniquely exotic charm. #xinjiang', '#CapCut #foryou #news #1minutenews #CapCut #foryou Storm in Tacheng, Xinjiang, China. (August 12, 2023).', '#Uyghur #China #Hunan #Xinjiang #Hunan #Uyghur #Uighur #Uyghur #EthnicCostumes #MinorityEthnicGroups #EthnicMinorityCulture #China #Xinjiang #Dance #UyghurDance #Uighur #Uighurs #Uyghurs', 'Beautiful Xinjiang Uyghur women striving in Shanghai #beautifulwomen #Chinese', "The heat in Xinjiang's Turpan is not exaggerated! #Xinjiang #xinjiang", 'This was taken during my trip to Xinjiang in China last year. I really want to make the most of the holiday to travel again! #holidaytravel #landscape #horseriding #travel #horseriding', 'Replying to @flowywer #dilrabadilmurat #fyp #tiktok #barbie #uighur #xinjiang #china #2023', "Come join us in celebrating the Chinese Lunar New Year with an exhilarating opening dance at the CCTV Spring Festival Gala 2024 Kashi Sub-venue! Let's usher in the Year of the Dragon with unrestrained vigor and boundless happiness! #HappyChineseLunarNewYear #Xinjiang #springfestival #Gala", 'Welcome to the ancient city of Kashgar, Xinjiang, China. #china #freestyle #xinjiangchina #uyghur #travel #uyghurmuslims #fyp #ReTokforNature #xinjianguyghurs', 'Children in Xinjiang... #China #Indonesia #education', 'Check this out! Can you keep up with them? (Credit to Humar 328 on Douyin) #xinjiang #dance #kids #interesting', '#Xinjiang #XinjiangChina #XinjiangCotton #UyghurMuslimsLifeMatter #XinjiangReeducationCamps #XinjiangDance #Uyghur #XinjiangUyghurs #XinjiangA', 'The Altay region in Xinjiang is a charming paradise where wild nature beautifully blends with soft elegance in perfect balance. Prepare to be mesmerized by its stunning beauty! #Xinjiang #scenery', 'Winter in my hometown, Xinjiang', 'Animals in China are freezing due to low temperatures #2024 #viral #foryou #trending #china #xinjiang #newearth #5d #weather #news', 'Replying to @kctan673 #Xinjiang', 'Major historical and cultural monuments under state protection" nothing is being abolished! #Uyghur #Xinjiang #Muslimah #XinjiangChina #China #FYP #Uygur', '#travelchina #xinjiangchina #xinjiang #urumqi #airchina #hemuvillage #wintervibes', 'Xinjiang is a fantastic destination for a road trip. Traverse through the fascinating ancient city of Kashgar, savor the local food, and marvel at the snow-capped mountains. Visit Xinjiang to experience the traditions of the western region. #xinjiang #food', 'At a Uyghur wedding ceremony, the bride dances with elegance and happiness, celebrating love in the most captivating manner. #Xinjiang #wedding #dance #bride', 'Welcome to Xinjiang #foryourpage #china #xinjiang', '#Xinjiang #China #AUKUS #France #UighurMuslims #Afghanistan #Police', "Update: A 7.1-magnitude earthquake hits China's Xinjiang. #new #earthquake #china #hit #xinjiang #update #2024 #foryourpage", 'This is the Ken Swart water conservation project in Xinjiang. "Ken Swart" is a Mongolian term that means a place where water converges. It is also the longest and most striking Danxia landform found in China, referred to as the "Geographical Landscape of the Tianshan Mountains." #xinjiang', 'At 10:30 PM in Xinjiang, people just start going to the market to cook rice #travel #xinjiang #trend #RunawayPrincess #GreatTVShow', 'Xinjiang, China #scenery #tiktok #foryoupage #beautiful #china', 'The Most Famous Bread in Xinjiang, China #china #xinjiang #urumqi', "You have to check out the dancing boys in Xinjiang! It's a unique and captivating experience that you have to witness if you ever get the chance to visit Xinjiang. #Xinjiang #DancingBoys #Tradition #Energetic #Charm #Talent #fyp #boys", 'We took a short trip to Xinjiang for a few days! How do we look in these outfits? (Xinjiang part.1)', "Don't travel to Xinjiang by yourself, let alone visit Selim Lake alone, because the beauty will move you to tears and there will be no one to wipe them away for you...! #Scenery #Nature #Water #China #tiktok #ice #xinjiang", 'Members of the delegation from the Arab League who visited Xinjiang from May 30 to June 2. China welcomes friends from all countries to come and experience firsthand a prosperous, stable and harmonious Xinjiang.', 'The heat in Xinjiang Turpan is not exaggerated! #Xinjiang #xinjiang', '#asmr #bbq #charcoalbbq #xinjiang #lambskewers #masterbuilt #bbqrecipe #foryoupage', 'The atrocities of Xinjiang, Part 0. #ForYou #ForYou #TikTok #ForYou #FYP #FYI #FYPPage #ForYou #Real #Reality #ChinaTikTok #China #Xinjiang #XinjiangChina', 'A family of three came to take Chinese ID photos #Xuke #FengMing #KaiXinJiang #XuMuyan #ForYourPage #Trending #RecommendedForYou', 'Check this out! Can you keep up with them? (Credit to Humar 328 on Douyin) #xinjiang #dance #kids #interesting', "She's so beautiful! Dilraba is dancing a traditional Uyghur dance at the Chinese Spring Festival 2024 in Xinjiang. #fyp #foryou #foryoupage #beauty #girl #dance #tiktok #chinese #spring #festival", 'Interview with a beautiful girl in Xinjiang #foryourpage #beautiful #beauty #interview #pure', "Is there a Chinese Emperor who practices Islam? Let's Discuss What else do you want to discuss about Chinese culture? ExcitingLearning IslamicHistory Xinjiang MingDynasty OscarDarmawan", "The leader of the World Muslim Communities Council visited the Xinjiang Uygur Autonomous Region in China and commended China's efforts in preserving ethnic cultures. #muslimculture", "Update: A 7.1 magnitude earthquake hits China's Xinjiang. #new #earthquake #china #hit #xinjiang #update #2024 #fyp", 'The Id Kar Mosque in Xinjiang is the largest mosque in Xinjiang. Constructed in 1442, it still accommodates up to 10,000 worshippers every Friday. #mosque #masjid #muslim #islam #prayer #ramadan #xinjiang', "POV: You're traveling in Xinjiang, China. Did you know, buddies, Xinjiang is an autonomous region in China that has a unique geographical location as it borders Tibet, Qianghai, and Gansu. Not only that, Xinjiang also borders Mongolia, Russia, Kazakhstan, Kyrgyzstan, Tajikistan, Afghanistan, and even Kashmir. Literally, the meaning of XinJiang is 'New Border' or 'New Region'. This name was given during the Manchu Qing Dynasty. Credit: Annie Lin-En #xinjiang #xinjiangchina #xinjiangtravel #chinatravel #explorechina #visitchina #china #chinatrip", 'The recent dance craze in China, "#Kemusan", reflects the joyful lifestyle of the Chinese people. This joy is expressed through the elegant movements of Xinjiang performers to the people of Qatar, enthralling everyone who sees it! #2024XinjiangCulturalandTourismPromotionWeek #Kemusan #XinjiangMusicandDance #XinjiangIsaNicePlace #XinCulture #ThisIsXinjiang #DreamyXinjiang', 'When is the best time to visit Xinjiang? If you have any other questions, leave a comment below! #xinjiang #travellife #traveltochina #mustvisitplaces', 'Beautiful Uyghur nurse at the hospital in China #foryoupage #Uyghur #UyghurTiktok #UyghurMuslims #UyghurWoman #Xinjiang #China', "Here's a travel tip: You absolutely must experience Xinjiang in the fall! See for yourself the crystal-clear waters, the majestic mountains, and forests dressed in vibrant colors. It's a sight you can't miss! #Xinjiang #Autumn #Kanas #mountains #travel #fyp #nature", 'Welcome to Xinjiang, the home of the Uighur people in China. #islam #muslim #uighur #uighurmuslims #china #chinesemuslim', '#Bagua City, #Xinjiang     #Bagua City, Xinjiang', 'I miss the Meiji factory in Xinjiang. #meijimill #xinjiang #cotton', 'This is what Urumqi in Xinjiang looks like #xinjiang #urumqi #china', 'Exploring a hidden gem in Xinjiang! Join me as I wander through this charming village, surrounded by majestic mountains, serene river streams, and lush greenery. #Xinjiang#XinjiangVillageLife#NatureBeauty#PeacefulRetreat#AdventureLife', 'Natural Kashgar dance, China, travel, Xinjiang China, for your page, Uyghur Muslims, Xinjiang Uyghurs, Uyghur, Xinjiang, fitness motivation.', 'The Gobi Desert highway in Xinjiang is covered with sand #xinjiang #xinjiangchina #xinjiangtravel #china #gobi #chinatravel #gobidesert.', 'Kashgar in Xinjiang #Xinjiang #Kashgar #Uyghur #China #Travel', "Part 1: Compiled from recommendations from locals, here are the top 10 dishes you must try in Xinjiang. Missing out on these means you've missed out on everything! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltips #musteatfood #viral #fyp", '#beauty #girl #XinJiang #wear', 'Urumqi, my love! Have you witnessed the magic of the sunrise here? #Urumqi #SunriseMagic #Xinjiang #city #fyp #rotateyourphone', '#China #Xinjiang #Urumqi #vlogger #UK #Uyghur #xyzbca', 'Breathtaking places in Xinjiang. #work #scenery #China #foryourpage #foryou #goingviral #foryoupage #Xinjiang', 'This is winter in Xinjiang. Come and have fun. #scenery', 'Immerse yourself in the snowy paradise of Xinjiang and experience a winter wonderland like never before! #WinterMagicInXinjiang #snowworld', '#Xinjiang #Kazakh girl', 'This is not New Zealand or Switzerland. This is the untold story of Xinjiang, the great homeland of Chinese Muslims. Follow for more on the grand rivers and mountains of Xinjiang. #MuslimTourism #UyghurMuslims #Uyghur #HuiMuslim #ChinaTravel #ChineseMuslimTourism #ChineseMalay #FYP #TravelChina #ChineseMuslim #MuslimAroundTheWorld #Xinjiang #TravelMalaysia #TravelTikTok', "Xinjiang's natural beauty introduces us to charming animals that are typically found in zoos. Can you resist being mesmerized by the captivating look of these little foxes? #Xinjiang #Fox", 'Xinjiang. #beautifulview #foryou #cure #sceneryvideos #nature', 'Replying to @someone_from_the_street #urumqi #xinjiang #xinjiangchina #uyghurmuslims #uyghurlivesmatter #china #chinatiktok #prc #cpc #not #ccp #mrfong #politics #geopolitics #politics #politicaltiktok #fy #foryoupage #foryou #foryourpage #foryou', 'White, the color of purity and divine illumination, cleanses our minds and brings us peace. #Xinjiang #colors #white #colorfulXinjiang #snow #ice #clouds #scenery #nature', 'Deadly sea urchin hairstyle #foryou #fyp #seaurchin #turpan #xinjiang', 'A paradise for food lovers! The variety here exceeds your imagination! #xinjiang #xinjiangtravel #xinjiangfood #lambkebab #bbq #yogurt #nightmarket #foodie #chinesefoodlover #vlog #fyp', 'Responding to @nursafiahd guys from Xinjiang, um, let the ladies comment haha #xinjiang', 'November 28th. Many sheep and cattle are freezing to death due to a snowstorm in Xinjiang, China. #snow #snowstorm #china', 'Urumqi Grand Bazaar China Xinjiang', "Xinjiang Emerald Raisins, dried green grapes. It's a western specialty that tastes sweet and delicious. #raisins #grapes #Russian #drinks #shorts #snack", 'Policewomen from the Xinjiang Uighur region were invited to dance together, debunking Western rumors of China committing genocide. The 56 ethnic communities in China, collectively referred to as the Chinese nation, have been united for thousands of years. The slaughter of Native Americans in the United States is the actual genocide. #china #xinjiang #police', 'Handsome guy from Xinjiang', 'Experience the Vibrant Red River Valley in Toksun county, Xinjiang! Immerse yourself in the breathtaking yardang landforms, lush green landscapes, and majestic mountains. #Tianshan #Turpan #fyp #Xinjiang #NatureBeauty #ScenicSpot #travel #adventure', 'Ice pushing, travel, China, Sayram Lake, Xinjiang, ice push.', 'China uses "Uighur" influencers to promote its regime. #China #Uighurs #Influencer #Xinjiang #SocialNetworks #Daily', 'Interview with a 5\'10" beauty from Xinjiang #beautiful #beauty #beautifulwoman #Asian #idolface #goddess #beauty', "Join us to celebrate the Chinese Lunar New Year with an exhilarating opening dance at the CCTV Spring Festival Gala 2024 Kashi Sub-venue! Let's commence the Year of the Dragon with infectious enthusiasm and boundless happiness! #HappyChineseLunarNewYear #Xinjiang #springfestival #Gala", '#Xinjiang #China #SaveXinjiang #ForYourPage', 'Responding to @ChicoCerdito about the #greenscreen on #SocialCredit and #Xinjiang and US media', 'Welcome you to travel to the ancient city of Kashgar, Xinjiang #Uyghur #Uyghur #XinjiangChina #Xinjiang #XinjiangUyghurs #Xinjiang #UyghurMuslims #Travel', "A snow removal fleet is clearing a street in China's Xinjiang.", 'My grandpa looks very happy, haha. For your page, Xinjiang, Xinjiang China, travel TikTok.', 'If you visit Xinjiang, you will run across the grasslands, dance beside the lakes, sing in the mountains and fulfill all your desires in this paradise-like destination. #Xinjiang #travel #havefun', 'What if Xinjiang was a Country #geography #countries', 'In this barren place of Karamay, Xinjiang, it was discovered that the underground is full of oil fields #scenery #china #tiktok #fyp #foryou', "Participate in the lively festivities where friends from Kashgar in Xinjiang are taking part in the #LoongYearLoongDance challenge. Let's bond in enjoyment and dance into a year full of wealth and joy!", '#Uyghur #Xinjiang #China #USA #Europe #Huachunying #HuaChunying', 'My surname is Ka Ka Shi Xinjiang xinjiang Kashgar', "Doctors in #China have shown their courage in facing the call of their profession amid adversity. This is after they continued to perform brain surgery amid an earthquake. They successfully completed the life-saving craniotomy surgery while the Xinjiang's Aral Hospital was being shaken by a 7.1 magnitude earthquake. #News5 #NewsPH #SocialNewsPH", "12 hours at a rapper's wedding in China! (in Xinjiang) #china #xinjiang #rapper #asia #uyghurs #scorinchina", 'Known as the "capital of the geographical center of the Asian continent", Urumqi, Xinjiang, is the world\'s largest city farthest from the ocean. #Xinjiang #city', 'I enrolled in an English class in Xinjiang... Have you ever wondered what the schools in Xinjiang are like? #xinjiang #education #students #primaryschool #englishteacher #kids #uyghur #chinatiktok', '#beauty #girls #dance #XinjiangChina', "Lost in the hug of majestic mountains, Xinjiang's Altay reveals a hidden paradise. A peaceful sanctuary tucked away amidst the wonders of nature. #Xinjiang #nature", '#Xinjiang #China #Intel', 'Would you like to try? Please follow my comment #travel #xinjiangchina #snow #beautiful #fyp', "What a thrilling sight of a plentiful harvest! In Xinjiang, China's largest high-quality cotton production base, over 6,900 cotton-picking machines roar into action across a vast stretch of cotton fields. They present a splendid scene and finish the picking task efficiently! The diversity of cotton species in Xinjiang includes Fine Staple cotton, Upland Cotton, and Long-staple Cotton. They have also naturally bred Colored Cotton. In total, Xinjiang's cotton yield makes up about 20% of the world's cotton production. The cotton industry in Xinjiang has developed into an efficient, energy-saving, and high value-added integrated industry chain, encompassing every stage from cotton planting, processing, and cotton trading, to dyeing, spinning, and weaving. China's cotton industry is burgeoning, flourishing, and progressing in Xinjiang. And it is heading towards a brighter future! #insiders #XJonair #diveintoXJ #cotton #machine #industry", 'How people celebrate and welcome New Year 2023 in Xinjiang with a large flame #xinjiang #xinjiangchina #hello2023 #china #newyear #happynewyear', 'Leaked police documents have exposed the identities, including names and photos, of over 2,800 Uyghurs, a predominately Muslim ethnic minority who are being held captive by the Chinese government in Xinjiang. Many of them were imprisoned based on questionable charges. Some Uyghurs who have escaped to other countries discovered their lost family members listed in the database. #uyghur #china #muslims', "Experience the excitement of sand skiing in the expansive desert of Turpan, Xinjiang! Who's ready for a desert adventure? #xinjiang #desert", "At 8 o'clock, Xinjiang's specialties remain the same #foryoupage", 'What if Xinjiang becomes an independent country | Country Comparison #tips', 'The Karakoram Highway, also known as the China-Pakistan Friendship Highway, is a 1,300-kilometer-long highway that links Pakistan and China. This highway is a popular destination for tourists and stands as one of the highest paved roads globally, reaching a peak elevation of 4,714 meters (15,466 feet). #travel #ForYourPage #Xinjiang #China #Pakistan #Pamir #Karakoram #highway #road #driving #scenery #plateau', 'Conduct an interview with a student from Xinjiang #interview #student #havu2904', 'Jerry Good shares the truth about Xinjiang during his visit there...  #china #xinjiang', 'On the way to the top, sunshine, snow, rain, wind, all in one. On the way to Duku Highway #Xinjiang #China', 'Xinjiang is truly a lovely place. Come and visit Xinjiang! #foryourpage #tiktoktravel', 'Beautiful Uyghur women, Xinjiang, China, Xinjiang China, Uyghur, Muslim, Muslim woman, Turkmenistan, Xinjiang Uyghur, 365 days quarantine, for you, FY.', 'On October 8, the Fangcai facility in Xinjiang was under construction. #China #ForYourPage #Isolation #ConcentrationCamp #NewWorldOrder #GreatReset.', 'Interview with a girl from Xinjiang #ForYourPage #beautiful #beauty #pretty #pure #interview', '#Wind-bell costume #china #hunan #xinjiang #chinese #Uighur #ethnicminority #China #ethnicclothing #uighur #uyghur #xinjiang #hunan #beijing #UighurGirl #beijing #whatareyouasian #uyghur #uighur', "Girls from Xinjiang are natural-born dancers! The young girl's elegant movements match the music perfectly, displaying her distinctive dance skills. Get ready to be captivated! #Xinjiang #dance #talent #cutegirl", 'How vast is Xinjiang? Why is Xinjiang not free to ship? #china #trungquoc #tancuong #xinjiang', 'Largest mosque in China, located in Xinjiang province - IDKAH Mosque - hashtag being a traveler', 'The atrocities of Xinjiang, Part 6. #urgent #emergency #world #worldtiktok #worldwide #foryou #foryoutiktok #tiktok #foryourpage #foryourpage #fypage #fyp #fyi #fypシ #fypツ #fypageシ #video #viralvideo #foryou #viraltiktok #chinatiktok #holocaust #uyghurs #uyghurpeople #uighurs #china #xinjiang #women #8m #feminism #terrortiktok #terror', 'KFC, Xinjiang, China, BBC, Uighur, UK, vlogger, for your page', 'Xinjiang, an enigmatic region situated in Northwest China, is abundant with unique allure and natural beauty. The mountains, deserts, grasslands, and glaciers there compose a vibrant tableau of nature. The stunning beauty of Xinjiang is simply awaiting your discovery. #xinjiang', 'THE HUI (CHINESE MUSLIM) UIGHUR TRIBE IN XINJIANG SOMEWHAT RESEMBLE "CAUCASIANS", WHY? Surely, there must be some who have also wondered, why do Xinjiang people have different faces compared to Chinese people in general. Turns out there\'s history behind this! Now you know, right? Don\'t forget to share, so your other friends also know, okay! #xinjiang #chineseculture #chinatownculture #learningmandarin #mandarinlanguage #learnmandarin #mcmandarin', '#Uyghur #foodtiktok #cuisine #Uyghur #xinjiangchina #Xinjiang #foryoupage #foryou Xinjiang ramen 10 yuan each', "She's so beautiful! Dilraba is dancing a traditional Uyghur dance at the 2024 Chinese Spring Festival in Xinjiang. #foryou #foryoupage #beauty #girl #dance #tiktok #chinese #spring #festival ", 'Exploring a hidden gem in Xinjiang! Join me as I wander through this charming village, surrounded by majestic mountains, serene river streams, and lush greenery. #Xinjiang #XinjiangVillageLife #NatureBeauty #PeacefulRetreat #AdventureLife', 'Interview with a 1.78M tall beauty from Xinjiang #beautiful #beauty #beautifulwoman #Asian #idolface #goddess #beauty', 'Dive into this golden world, where autumn in Kanas, Xinjiang stands out incomparably. The awe-inspiring beauty is worth the long journey. #xinjiang #scenery #autumn', "Laughing and I'm about to collapse #grandepicune #XinjiangBeauty #HighSchoolGirls #AttractiveStudents #XinjiangStudents #BeautifulGirls #XinjiangHighSchool #StudentBeauty #XinjiangGirls #AttractiveHighSchoolers #XinjiangCharm #HighSchoolBeauty #XinjiangWomen #StudentCharm #XinjiangTeens #AttractiveTeens #XinjiangYouth #HighSchoolCharm #XinjiangLadies #StudentGrace #XinjiangFemales #BeautyOfXinjiang #MiddleSchoolStudents #AttractiveStudents #StudentsOfXinjiang #BeautifulGirl #MiddleSchoolOfXinjiang #BeautyOfStudents #GirlsOfXinjiang #AttractiveMiddleSchoolStudents #CharmOfXinjiang #BeautyOfMiddleSchool #WomenOfXinjiang #AttractivenessOfStudents #YouthOfXinjiang #AttractiveYouth #YouthOfXinjiang", 'Interview with a beautiful girl in Xinjiang #foryourpage #beautiful #beauty #interview #pure', '#beauty #girls #dance #xinjiangchina', 'Part 1: Top 10 Treasures of Xinjiang - A Must-Visit to Uncover the Beauty of Xinjiang #Xinjiang #TravelLife #TravelToChina #MustVisitPlaces #Viral #FYP', 'Experience the Vibrant Red River Valley in Toksun county, Xinjiang! Get ready to explore the awe-inspiring yardang landform, lush greenery, and majestic mountains. #Tianshan #Turpan #foryourpage #Xinjiang #NaturalBeauty #ScenicSpot #travel #adventure', '#beauty #girl #XinJiang #wear', "Hi there! Today, I want to talk about one of the most fascinating countries in the world – China! China is a country with a rich history, vast culture, and breathtaking natural beauty. From the bustling streets of Beijing to the serene mountains of Tibet, China has something to offer everyone. One of the most striking features of China is its immense size. With an area of 9.6 million km², it is the world's second-largest country after Russia. However, despite its size, China is a nation of incredible diversity, with countless cultures and languages spoken across its vast landscape. \n\nChina is also known for its impressive achievements in technology, science, and engineering. From the world's longest suspension bridge to the world's largest LED screen, China has been at the forefront of innovation for centuries. But China is not just known for its technological advancements. It is also a country of breathtaking natural beauty. From the lush green valleys of the Himalayas to the stunning coral reefs of the Pacific Ocean, China is home to some of the world's most beautiful landscapes. \n\nOf course, no trip to China would be complete without trying some of its delicious cuisine. From spicy hot pot to fragrant stir-fry, Chinese food is a true culinary delight. And with an estimated 3.5 billion people around the world who love to eat, China has a huge global market for its food. \n\nOverall, China is a country that truly has something for everyone. From its stunning natural beauty to its impressive technological advancements, China is a country that is sure to captivate and inspire anyone who takes the time to explore it. I highly recommend a trip to China – you won't be disappointed! #china #chinese #cn #taiwan #edit #nameacountryforanedit #nameacountry #fun #military", "Don't travel to Xinjiang by yourself, and especially avoid visiting Selim Lake alone, because the beauty is so overwhelming it may bring you to tears and no one will be there to console you...! #Scenery #Nature #Water #China #tiktok #ice #xinjiang", 'Some beautiful girls from Xinjiang. #interview #beautifulwomen #beauty #beauty #fyp', 'Explore the mystical beauty of Altay, Xinjiang, and savor the moment as you watch one of the most breathtaking sunsets unfold before your eyes. #foryourpage #nature #winter', 'The beauty of Xinjiang is simply enchanting! When are you planning your trip? #Xinjiang #TravelGoals #travel #fyp #nature #mountains #forest', 'Mummy that maintains beauty... #Uyghur #China #science #history', 'The most beautiful Xinjiang scenery #nature #spring #beautiful #beauty #foryou #foryourpage #featured', 'Xinjiang, a mysterious region in Northwestern China, is abundant with unique allure and stunning natural beauty. Its mountains, deserts, grasslands, and glaciers paint a vibrant picture of nature. The scenic beauty of Xinjiang eagerly awaits your exploration. #xinjiang.', "Xinjiang's natural beauty introduces us to charming animals usually found only in zoos. Can you resist being mesmerized by the enchanting look of these small foxes? #Xinjiang #Fox", 'Interview with a girl from Xinjiang #foryourpage #beautiful #beauty #pretty #pure #interview', 'Replying to @hartantiyunida95 #DewiDilraba #DilrabaDilmurat #AsianBeautyTikTok #2023 #queen #goddess #beautiful #Xinjiang #Uighur #foryourpage #foryou #video #viral #tiktok', 'Altay in Xinjiang is a stunning paradise where wild beauty is in perfect harmony with gentle elegance. Prepare to be mesmerized by its impressive scenery! #Xinjiang #scenery', 'Interview with bestie from Xinjiang #beauty #beautiful #cute #lovely #girl #pretty woman #charm #Asian', "An expatriate from Israel visited cotton farms in Xinjiang. Thank you for exploring Xinjiang and interacting with local farmers. It's time to buy a ticket to Xinjiang #Uyghur.", '#Xinjiang #XinjiangChina #XinjiangCotton #UyghurMuslimsLifeMatter #XinjiangReeducationCamps #XinjiangDance #Uyghur #XinjiangUyghurs #XinjiangA', "What an exciting sight of a plentiful harvest! In Xinjiang, the most significant production base of high-quality cotton in China, over 6,900 cotton-picking machines rumble as they traverse the cotton field. This creates a grand display and achieves the picking task efficiently! Xinjiang has various cotton species, including Fine Staple cotton, Upland Cotton, and Long-staple Cotton. They have also naturally bred Colored Cotton. Altogether, Xinjiang's cotton output makes up about 20% of the worldwide cotton production. The cotton industry in Xinjiang has formed an effective, energy-saving, and highly profitable integrated industry chain, from cotton planting, processing, and cotton trading, to dyeing, spinning, and weaving. China's cotton industry is growing, flourishing, and evolving in Xinjiang, it's heading towards a better future! #insiders #XJonair #diveintoXJ #cotton #machine #industry.", "What an exciting view of a plentiful harvest! In Xinjiang, the largest producer of high-quality cotton in China, more than 6,900 cotton-picking machines thunder as they traverse the cotton ocean. They create a stunning show and complete the picking task efficiently! Xinjiang's cotton varieties are diverse, including Fine Staple cotton, Upland Cotton, and Long-staple Cotton. It has also naturally cultivated Colored Cotton. In total, Xinjiang’s cotton production makes up about 20% of the worldwide cotton production. The cotton industry in Xinjiang has formed an efficient, energy-saving, and highly value-added integrated industry chain, running from cotton planting, processing, and trading, to dyeing, spinning, and weaving. The cotton industry in China is growing, flourishing, and transitioning in Xinjiang. And it is progressing towards a brighter future! #insiders #XJonair #diveintoXJ #cotton #machine #industry", 'So, any problem with Xinjiang Cotton? #China #Chinese #mandarin #language #study #learn #ChineseLanguage #MandarinLanguage #cotton #Xinjiang #Uyghur #farm #bilingual #vicali', 'Xinjiang, China #scenery #tiktok #foryoupage #beautiful #china', "Hello! Today, I want to discuss one of the most fascinating countries in the world - China! China is a country with a rich history, abundant culture, and breathtaking natural beauty. From the bustling streets of Beijing to the tranquil mountains of Tibet, China offers something for everyone. One of the most remarkable characteristics of China is its enormous size. With an area of 9.6 million km², it is the world's second-largest country, after Russia. However, despite its size, China is a nation of incredible diversity, with numerous cultures and languages spoken across its extensive landscape. \n\nChina is also distinguished by its impressive accomplishments in technology, science, and engineering. From the world's longest suspension bridge to the largest LED screen, China has been leading in innovation for centuries. Yet, China isn't only known for its technological advancements. It's also a country of magnificent natural beauty. From the lush green valleys of the Himalayas to the stunning coral reefs of the Pacific Ocean, China houses some of the world's most beautiful landscapes. \n\nOf course, no visit to China would be complete without tasting some of its delectable cuisine. From spicy hot pot to fragrant stir-fry, Chinese food is a true culinary delight. And with an estimation of 3.5 billion people worldwide who love to eat, China has a massive global market for its food. \n\nIn conclusion, China is a country that genuinely offers something for everyone. From its awe-inspiring natural beauty to its remarkable technological advancements, China is a country that will fascinate and invigorate anyone who takes the time to discover it. I highly recommend a trip to China - you won't be disappointed! #China #Chinese #Taiwan #Edit #NameaCountryforanEdit #NameaCountry #Fun #Military", 'Would you like to try? Please follow my comment #travel #xinjiangchina #snow #beautiful #fyp', 'Explore the winter spectacle at Bohu County, Xinjiang where the beautiful Bosten Lake is located. This hidden gem captivates visitors with its breathtaking landscapes. #fyp #XinjiangSnow #nature #winter', 'Xinjiang. #beautifulview #foryou #cure #sceneryvideos #nature', 'Interview with best friend from Xinjiang #beauty #beautiful #cute #lovely #woman #beautifulwoman #pretty #Asian', 'Xinjiang, China #scenery #tiktok #foryourpage #beautiful #china', 'Interview with a beautiful girl in Xinjiang #foryourpage #beautiful #beauty #interview #pure', "She's so beautiful! Dilraba performing a traditional Uyghur dance at the Chinese Spring Festival 2024 in Xinjiang. For your page, beauty, girl, dance, TikTok, Chinese, spring, festival.", "The dancing and music traditions of Xinjiang are a lively depiction of the diverse cultural influences in the region. Ethnic groups like the Uygur, Kazak, Kyrgiz, and Tajik have contributed to this vibrant melting pot of cultures. The unique combination of traditional music and dance is a fascinating part of Xinjiang's rich cultural legacy. Thank you for sharing your beautiful culture with the world, Xinjiang! #Xinjiang #Uygur #Kazak #Kyrgiz #Tajik #traditionalmusic #dancing #culturalheritage #foryoupage #Yining", 'Interview with a girl from Xinjiang #foryourpage #beautiful #beauty #pretty #pure #interview', 'Beautiful Uyghur women #Xinjiang #China #XinjiangChina #Uyghur #Muslim #Muslimah #Turkmenistan #XinjiangUyghur #365DaysQuarantine #ForYou #FY', 'I feel like I\'m about to faint from laughing #bigwalgreens #XinjiangBeauty #HighSchoolGirls #AttractiveStudents #XinjiangStudents #BeautifulGirls #XinjiangHighSchool #StudentBeauty #XinjiangGirls #AttractiveHighSchoolers #XinjiangCharm #HighSchoolBeauty #XinjiangWomen #StudentCharm #XinjiangTeens #AttractiveTeens #XinjiangYouth #HighSchoolCharm #XinjiangLadies #StudentGrace #XinjiangFemales #TanCuongBeauty #HighschoolStudents #AttractiveSchoolStudents #TanCuongStudents #BeautifulGirls #TanCuongHighSchool #SchoolStudentBeauty #TanCuongGirl #AttractiveHighschoolStudents #TanCuongEnchantment #HighSchoolBeauty #TanCuongWomen #StudentAllure #TanCuongYouth #AttractiveYouth #TanCuongTeens"\n', 'The most beautiful scenery in Xinjiang #scenery #nature #spring #beautiful #beauty #foryou #fyp #f', 'Replying to @hartantiyunida95 #dewidilraba #dilrabadilmurat #asianbeautytiktok #2023 #queen #goddess #beautiful #xinjiang #uighur #fyp #foryou #video #viral #tiktok', 'Beautiful Xinjiang', 'Here is the beautiful healing landscape of Xinjiang, China.', 'Some beautiful girls from Xinjiang. #interview #beautifulwomen #beauty #beauty #fyp', 'Beautiful Uyghur female police officer at a market in Urumqi, China #ForYourPage #Uyghur #UyghurTikTok #UyghurMuslims #Police #PoliceOfficer #PoliceTikTok #China #XinjiangChina #XinjiangTravel', "The images speak for themselves. What is your opinion now? From a cultural standpoint, this is one of the most beautiful countries I've ever visited, and I'm confident that once you're here, you will understand what we're talking about. The culture, people, and places have captivated me, and I've learned something new on every trip. After six years of traveling in China, my mind has opened in many different ways. Would you like to travel to China?... #chinatravel #lifeabroad #travelguide #china", 'Beautiful Uyghur nurse at the hospital in China #foryourpage #Uyghur #Uyghurtiktok #Uyghurmuslims #uyghurwoman #Xinjiang #China', 'Beautiful Uyghur women, Xinjiang, China, Xinjiang China, Uyghur, Muslim, Muslim woman, Turkmenistan, Xinjiang Uyghur, 365 Days Quarantine, for you, FY', "#tiktok #winter #cold #trending #skill #happiness #driving #road #nature #amazing. It's so beautiful but be careful.", 'Interview with a 5\'10" beauty from Xinjiang #beautiful #beauty #beautifullady #Asian #idolface #goddess #charminglady', "Update: A 7.1-magnitude earthquake has hit China's Xinjiang region. #new #earthquake #china #hit #xinjiang #update #2024 #fyp", 'Interview with a girl from Xinjiang #fyp #beautiful #beauty #pretty #pure #interview', "She's so beautiful! Dilraba performing a traditional Uyghur dance at the Chinese Spring Festival 2024 in Xinjiang. #ForYourPage #Beauty #Girl #Dance #TikTok #Chinese #Spring #Festival", 'Xu Ke / Feng Ming with his daughter vlog, like father like daughter, so cute and pretty #xuke #xuke #fengming #kaixinjiang #xumuyan #xumuyan #happytalk #fyp #thelastimmortal #shenyin #foryou', "Update: A 7.1-magnitude earthquake has struck China's Xinjiang. #new #earthquake #china #hit #xinjiang #update #2024 #fyp", 'The Xiongnu Empire — East Asia\'s "Huns" #history #chinesehistory #hungarian #mongolia #china #chinese #chinatok #chinesetiktok #hungary #ancienthistory #ancienthistorytok #turkic #turk #turkey #kazakhstan #mongolian #steppe #eurasian #centralasia #centralasian #asia #asianhistory #eurasia #language #fyp #foryou #foryoupage #fypshi #handynasty #siberia', '#history #china #maozedong #xijinping #foryourpage #foryou', "Here's a travel tip: You must experience Xinjiang in autumn! See the crystal-clear waters, majestic mountains, and forests decorated in vibrant colors. It's a sight to behold! #Xinjiang #Autumn #Kanas #mountains #travel #fyp #nature", 'My grandpa looks very happy, hahaha #ForYourPage #Xinjiang #XinjiangChina #TravelTikTok', '#westkalimantanpost #postwestkalimantan #foryourpage #fyp #viral #fypage #viralvideo #viral #tiktokviral #video #development #China', "New highway under construction in China's largest desert #fyp (for your page) #china #chinatiktok #xinjiang #techtok", 'Escape to tranquility in Kanas, Xinjiang - a village where time seems to stop. Immerse yourself in peace, embrace nature, and experience the comforting rhythms of local life. #Kanas #VillageLife #Nature #Xinjiang #Journey #fyp #quiet #travel', 'Really enjoy dancing traditional dances from my hometown #dilraba #dilrabadilmurat #dilireba #drachin #fyp #fypã‚·', 'Uighur community || instagram: willykun #foryoupage #tiktoknews #uighur #uighurlivesmatter #uighurmuslims #uyghur #uyghurpeople #sharefacts #youmustknow', 'Interview with a beautiful girl in Xinjiang #for your page #beautiful #beauty #interview #pure', 'Would you like to try? Please follow my comment #travel #xinjiangchina #snow #beautiful #fyp', 'Islamic figures around the globe defend Xinjiang. #islam #islamic #arabtiktok #arabs #asia #muslim #fyp', "A Chinese man records his investigation into alleged Uyghur 'concentration camps' in a video #china #concentrationcamp #reeducation #buzzfeed #fyp #news", 'Please, Hangzhou is so great! This city has a rich cultural heritage, including various historical sites, beautiful parks, and unique local traditions. #SandyInChina #Hangzhou #HangzhouChina #HangzhouAsianGames #ChinaTravel #RecommendationForTravel #TravelingChina #LifeInChina #StudyInChina #StudyAbroadChina #StudyingAbroad #IAmInChina #AllAboutChina #ChinaUniversity #UniInChina #StudyInBeijing #TravelInChina #ChinaVibes #ChinaLife #BeijingLife #ChinaTravel#Fyp', "In this barren place of Karamay, Xinjiang, we discovered that it's full of underground oil fields. #scenery #china #tiktok #fyp #foryou", '#Xinjiang #China #SaveXinjiang #ForYourPage', "You must see the dancing boys in Xinjiang! It's a unique and captivating experience that you must witness if you ever have the opportunity to visit Xinjiang. #Xinjiang #DancingBoys #Tradition #Energetic #Charm #Talent #fyp #boys", '#westkalimantanpost #postwestkalimantan #foryoupage #fyp #goingviral #foryoupage #viralvideo #viral #tiktokviral #video #development #china', 'Hidden camps going viral, relating to Islam, Muslim, Uyghur. For your page, spread awareness, help.', 'Replying to @hartantiyunida95 #dewidilraba #dilrabadilmurat #asianbeautytiktok #2023 #queen #goddess #beautiful #xinjiang #uighur #fyp #foryou #video #viral #tiktok', 'The atrocities of Xinjiang, part 6. #urgent #emergency #world #worldtiktok #global #foryou #foryoutiktok #tiktok #foryoupage #foryourpage #fypage #fyp #fyi #fyp #fypage #video #viralvideo #foryou #viraltiktok #chinatiktok #holocaust #uyghurs #uyghurpeople # Uighur #china #xinjiang #women #8m #feminism #terrortiktok #terror', 'Deadly sea urchin hairstyle #foryou #fyp #seaurchin #turpan #xinjiang', 'Spotted Kim Kardashian in Xinjiang! Would you be interested in taking a look? #xinjiang #traveltochina #bigbottoms #traveltiktok #travellife #viralfood #roastedlamb #viralvideo #fyp', 'Major historical and cultural monuments are under state protection, nothing is being abolished! #Uyghur #Xinjiang #Muslimah #XinjiangChina #China #FYP #Uygur', 'The Xiongnu Empire - East Asia\'s "Huns" #history #chinese history #Hungarian #Mongolia #China #Chinese #chinatok #chinesetiktok #Hungary #ancienthistory #ancienthistorytok #Turkic #Turk #Turkey #Kazakhstan #Mongolian #steppe #Eurasian #centralasia #centralasian #Asia #asianhistory #Eurasia #language #fyp #foryou #foryoupage #fyplikely #handynasty #Siberia', "Replylying to @Delilah_Lina Law. A Genocide of Uighur Muslims is happening in Xinjiang, but the Islamic world has chosen China's money over China's Muslims. #sheistheone16 #silenceisviolence #uighur #saveuighurmuslims #corrupt #foryourpage #gravitasplus #wion #thumbsup #foryourpage #foryou", 'This is neither New Zealand nor Switzerland...This is the untold story of Xinjiang, the wonderful motherland of Chinese Muslims. Follow for more about the breathtaking rivers and mountains of Xinjiang. #Muslimtourism #UyghurMuslims #Uygur #HuiMuslim #ChinaTravel #ChineseMuslimTourism #ChineseMalay #FYP #TravelChina #ChineseMuslim #MuslimAroundTheWorld #Xinjiang #TravelMalaysia #TravelTikTok', 'The falsehoods that the Western world has been spreading about Xinjiang are slowly being debunked, one by one. #west #us #humanrights #propaganda #germany #german #volkswagen #fyp', 'The breathtaking scenery in Xinjiang for part 3 @Breathtaking scenery. #travel #scenery #adventure #nature #China #Xinjiang #foryoupage #fantasy', 'Too engrossed in role playing Want more Virtual Tours? #centralasia #travelguide #uyghur #xinjiang #bazaar #kanaslake #foryoupage #xyzbca #cultures #travel', '#KFC #Xinjiang #China #BBC #Uighur #UK #vlogger #foryourpage', 'The windiest road in XinJiang, China, has 639 bends within 30 kilometers. Dare to try? #fyp #sharpturns', 'Some beautiful girls from Xinjiang. #interview #beautifulwomen #beauty #beauty #foryourpage', 'Replying to @flowywer #dilrabadilmurat #fyp #tiktok #barbie #uighur #xinjiang #china #2023', '#ASMR #BBQ #charcoalBBQ #Xinjiang #lambSkewers #masterBuilt #BBQrecipe #foryourpage', "The place in the world that's farthest from the sea is in China! Everyone should visit it at least once! #xinjiang #travellife #traveltochina #mustvisitplaces #viral #fyp", 'Falling clouds in the mountain #scenery #foryourpage', 'Uighur community || Instagram: willykun #fyp #tiktoknews #uighur #uighurlivesmatter #uighurmuslims #uyghur #uyghurpeople #sharefacts #youmustknow', 'Xinjiang, China #scenery #tiktok #foryoupage #beautiful #china', 'A paradise for food lovers! The variety here exceeds your imagination! #Xinjiang #XinjiangTravel #XinjiangFood #LambKebab #BBQ #Yogurt #NightMarket #Foodie #ChineseFoodLover #Vlog #FYP', "At 8 o'clock, Xinjiang's specialties remain the same #foryoupage", '10 Days in Xinjiang: A Journey Beyond Imagination! From the vibrant Urumqi to the enchanting Kanas, join us for an adventure like no other. #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp', 'Vibrant Red River Valley in Toksun county, Xinjiang! Explore the awe-inspiring yardang landform, lush greenery, and majestic mountains. #Tianshan #Turpan #fyp #Xinjiang #NatureBeauty #ScenicSpot #travel #adventure', 'Who wins? #Greece #GreeceTikTok #China #Chinese #ChineseFood #Food #FYP', 'Embark on a journey into the mystical beauty of Altay, Xinjiang, and savor the moment as you observe one of the most stunning sunsets unravel before your own eyes. #foryourpage #nature #winter', "History repeats itself. Uyghur lives matter, as do all lives regardless of religion or race. This conflict recalls memories of the Second World War, specifically the treatment of Jewish people, which is unjust. Muslims are now suffering similar persecution. Concentration camps are a dark part of history that should not be repeated. Agitated by these events, America and the rest of the world must take notice and act. Let's make this viral for everyone to see and act upon.\nFor you, let's create a viral wave of awareness using social media platforms like Tiktok. Please help this cause go viral by liking and sharing this post. Let's end the cycles of war, because history should not continuously repeat itself.\n", 'Took a chance and traveled to Xinjiang, where I found the meaning of life. Our group tour in Xinjiang is now available! Come and ask about it today! #xinjiang #travellife #traveltochina #mustvisitplaces #spring #springscentsation #viral #fyp', 'Step into the Narat grassland of Xinjiang~Pure joy and limitless awe encapsulated in every step. #travel #tour #China #Xinjiang #Narat #!China #acrossChina #wonderfulChina #amazingChina #WowChina #like #follow #fyp', 'Major historical and cultural monuments under state protection" - nothing is being abolished! #Uyghur #Xinjiang #Muslimah #XinjiangChina #China #fyp #Uygur', 'Unable to choose between Southern or Northern Xinjiang? Why not visit both! Take a look at our 14-day schedule that covers both regions of Xinjiang! #xinjiang #china #traveltips #travellife #traveltochina #mustvisit #vacation #scenery #fyp #viral', 'Good night, Urumqi. #fyp #Xinjiang #Urumqi #urumqi_xinjiang #sunset #goodnight #city', 'Natural Kashgar dance#China #Travel #XinjiangChina #ForYourPage #UyghurMuslims #XinjiangUyghurs #Uyghur #Xinjiang #FitnessMotivation', 'The breathtaking scenery in Xinjiang for part 6 @Breathtaking scenery. #travel #scenery #adventure #nature #China #Xinjiang #foryourpage #fantasy', "Here's our 10-day itinerary for Xinjiang! If you like it, let's plan a trip together next year! Feel free to get in touch with us and we can organize the Xinjiang trip you've been dreaming of! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", 'The atrocities of Xinjiang, part 0. #ForYou #foryou #tiktok #foryoupage #fyi #fypage #forYou #real #reality #chinatiktok #china #xinjiang #xinjiangchina', "Credibility of BBC? China's spokesperson Huachunying talks about media and Xinjiang #foryoupage #foryou", 'Traditional, characteristic wedding in Xinjiang Uyghur (China) #Xinjiang #China #Uyghur #XinjiangUyghur #ForYourPage', '"Children from Xinjiang bring pure joy to the Forbidden City with their captivating dance! #foryourpage #viral #trending #dance #chinese #tiktok"', 'Xinjiang, China #scenery #tiktok #foryoupage #beautiful #china', 'Debunking lies about China #foryoupage #fakenews #bbc #uygur #china', 'Xinjiang China Xinjiang is a place where many Muslim people live #China #xinjiangchina #fypp #inchina #fyp', 'Beautiful Uyghur female police officer in a market in Urumqi, China #foryourpage #Uyghur #UyghurTikTok #UyghurMuslims #Police #PoliceOfficer #PoliceTikTok #China #XinjiangChina #XinjiangTravel', '#Uyghur #foodtiktok #food #Uyghur #xinjiangchina #Xinjiang #fyp #foryourpage Xinjiang ramen 10 yuan each', '#greenscreen #foryourpage #uyghurmuslims #uygur #press', 'Tibetan boys #Tibetan #nomad #cowboy #lifestyle #fyp #centralasia #uyghur #xinjiang #freetibet #tibet', 'Urumqi, my love! Have you witnessed the magic of the sunrise here? #Urumqi #SunriseMagic #Xinjiang #city #fyp #rotateyourphone', 'On October 8, the Xinjiang Containment (Fangcai) was under construction #china #foryourpage #isolation #concentrationcamp #newworldorder #greatreset', 'Dancing! #Xinjiang #Uyghur #Uygar #Muslim #Muslimah #China #365DaysQuarantine #XinjiangChina #XinjiangUyghur #ForYourPage #ForYou', 'Welcome to Xinjiang #fyp #china #xinjiang', "Let's see how deep the snow in Xinjiang is~ #snow #heavysnow #tiktok #like #fyp #xinjiangchina #xinjiang", 'Xinjiang is truly a wonderful place. Come and visit Xinjiang! #foryourpage #tiktoktravel', 'A family of three came to take Chinese ID photos #xuke #徐可 #fengming #kaixinjiang #xumuyan #徐沐婵 #沐言开心酱 #fyp #foryoupage #foryou #fypselect #fy', 'Explore the winter spectacle at #Bohu county, #Xinjiang where the beautiful Bosten Lake is located. This hidden gem captivates visitors with its breathtaking landscapes. #fyp #XinjiangSnow #nature #winter', "The dancing and music traditions in Xinjiang are a lively representation of the region's multiculturalism, having been influenced by various ethnic groups such as the Uygur, Kazak, Kyrgiz, and Tajik. The exceptional mix of traditional music and dance is a fascinating aspect of Xinjiang's vast cultural heritage. Thank you for sharing your beautiful culture with the world, Xinjiang!", 'The last place is shocking. #travel #scenery #adventure #nature #China #Tianjin #fyp #fantasy', 'What is there to do in Xinjiang in autumn? #learnUyghur #china #Xinjiang #uyghur #xinjianguyghur #foryourpage #xinjianguyghurs #EidAlAdha', 'Breathtaking places in Xinjiang. #work #scenery #China #fyp #foryou #fypviral #fyp #Xinjiang', 'Beautiful Uyghur nurse at the hospital in China #fyp #Uyghur #Uyghurtiktok #Uyghurmuslims #uyghurwoman #Xinjiang #China', 'The most winding road in XinJiang, China, 30 kilometers with 639 turns. Dare to try? #fyp #sharpturns', 'The Karakoram Highway (also known as the China-Pakistan Friendship Highway) is a 1,300-km long highway connecting Pakistan and China. This highway serves as a popular tourist spot and ranks amongst the highest tarred roads in the world, reaching a peak elevation of 4,714 m (15,466 ft). #travel #foryourpage #Xinjiang #China #Pakistan #Pamir #Karakoram #highway #road #driving #scenery #plateau', 'Girls dancing at Uyghur weddings in Xinjiang, China, are freer and more secular than Muslims in the vast majority of the world! #UyghursMuslims #UyghurSavePlace #SaveTheUighurs #RealityOfXianjiangConcentrationCamps #XinjiangconcentrationCamp #SaveUighurs #UighurHumanRights #HelpTheUighurs #NoToUighurPersecution #RescueUighurs #Uighurs #FreeUighurs #Xinjiang #XinjiangRussia #UighursLove #UighursAreCaptivatedByTikTok #UighursFire #Uighurcuties #UighursInChina #UighursInTurkey #FreedomForUighurs #UighursAreNotAlone #EastTurkestan #EastTurkestanflag #ChinaChinese #ChinaFrance #UighursArabic #UighursArabicNation #ChinaArabic #ChinaArabicClap #ForYou #ForYouPage #ForYouViral #ForYourPage', 'There is not any literal translation for these hashtags as they are related to specific topics. In context, it indicates the following topics or terms: "KFC", "Xinjiang (a region in China)", "China", "BBC (British Broadcasting Corporation)", "Uighur (an ethnic group in China)", "UK (United Kingdom)", "Vlogger (someone who posts blogs in video format)", and "FYP (For Your Page, a tag commonly used on TikTok)".', "Part 1: Compiled from the suggestions of locals, these are the top 10 dishes you must try in Xinjiang. Failing to sample these means you've truly missed out! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltips #musteatfood #viral #fyp", 'Natural Kashgar dance from China. Travel to Xinjiang, China. A trending post for the Uyghur Muslims and Uyghurs from Xinjiang. Tag Uyghur and Xinjiang. Fitness motivation.', 'What do you think of this boy? #fyp #interview #foryou', '#Xinjiang #China #SaveXinjiang #ForYourPage', 'His nose is really adorable, darling #dilrabadilmurat #dilraba #dilireba #fyp #fypviral', 'Chinese Uyghur Muslim dress in Xinjiang #fyp #Uyghur #UyghurTiktok #UyghurMuslims #ChineseMuslims', 'Part 1: Top 10 Gems of Xinjiang - A Must-Visit to Uncover the Beauty of Xinjiang #xinjiang #travellife #traveltochina #mustvisitplaces #viral #fyp', 'The last place is shocking. #travel #scenery #adventure #nature #China #Hebei #fyp #fantasy', 'Nightlife atmosphere in Urumqi, Xinjiang Uygur Autonomous Region, China #urumchi #urumqinightlife #xinjiangnightlife #xinjiang #freeuyghurs #freeuyghurs #freeuyghurs #chinatiktok #china #chinesetiktok #nightlifeurumchi #urumchi #nightlifexinjiang #dunganpeople #UyghursinChina #FreeUighurs #uygurs #freedomforuygurs #uygurcauseinchina #Urumqi #UrumqiNightlife #Urumqi #UrumqiEmoji #foryou #foryourpage #foryoupage #Viral', "Don't travel overseas, because visiting Xinjiang is like experiencing the entire world in one place! #xinjiang #travellife #traveltochina #mustvisitplaces #viralvideo #fyp", 'The most beautiful scenery in Xinjiang #scenery #nature #spring #beautiful #beauty #foryou #fyp #f', "This is our amazing travel guide for Northern Xinjiang! Get in touch with us directly, and we'll help you plan everything! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", "This is our incredible travel guide for Northern Xinjiang! Reach out to us directly, and we'll assist you in organizing everything! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", 'and the list continues #for your page #muslim woman who wears hijab #muslim #muslim lives matter".', "This pack isn't bothered by the snow anyway. #TheWeatherChannel #fyp #weather #snow #winter #wolf #wolves #animals", 'Why does the US want to separate Xinjiang from China? #us #china #islam #muslim #asia #middleeast #geopolitics #fyp', 'Replying to @someone_from_the_street #urumqi #xinjiang #xinjiangchina #uyghurmuslims #uyghurlivesmatter #china #chinatiktok #prc #cpc #not #ccp #mrfong #politics #geopolitics #politics #politicaltiktok #fy #fyp #foryou #foryourpage #foryou', 'The Mountain of Fire, which even the Monkey King can\'t cross, is here?!#china #foryou #fyp #tiktok".', 'Life of Uighur Muslims in Xinjiang, Mainland China #chinesemuslim #muslimuighur #chinamuslim #china #trendingvideo #trendingtiktok #fyp #chinatiktok #muslimhui #ramadan2023 #ramadan #ramadanmubarak #ramadanmubarak2023 #ramadhanmubarak2023 #ramadan2023inshallah', 'Welcome to the ancient city of Kashgar, Xinjiang, China. #China #Freestyle #XinjiangChina #Uyghur #Travel #UyghurMuslims #FYP #ReTokForNature #XinjiangUyghurs #".', 'The beauty of Xinjiang is simply captivating! When are you planning your trip? #Xinjiang #TravelGoals #travel #fyp #nature #mountains #forest', 'The ending is touching #TikTokVoiceFeature #ChineseMuslim #Muslim #China #HalalFood #XinjiangFood #Culinary #Viral #ViralVideo #ViralTikTok #fyp #fypage #fypsounds', "Did you know that Xinjiang has both Southern and Northern regions? Check out this video to understand more quickly! Let's discover it together! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltips #viral #fyp", 'Turkish with Uighurs #travel #travel #tours #uyghur #china #xinjiang #eastturkistan #fy #fyp', '#westkalimantanpost #postwestkalimantan #foryourpage #fyp #foryourpageviral #fypage #viralvideo #viral #tiktokviral #video #development #china', 'Girls dancing at weddings of Uyghurs in Xinjiang, China, are freer and more secular than Muslims in the vast majority of the world! #uyghursmuslims #uyghursazmakani #SaveTheUyghurs #XinjiangInternmentCamps #XinjiangConcentrationCamps #SaveUyghur #UyghurHumanRights #HelpTheUyghurs #DoNotAllowUyghurPersecution #WantToSaveUyghurs #Uighurs #FreeUyghurs #xinjiang #sintzyan #uyghurs #UyghursWeSupportYou #UyghurBeauties #UyghursInChina #UyghursInChina #FreedomForUyghurs #UyghursYouAreNotAlone #EastTurkestan #China #Uighurs #fy #fyp #fypシviral #fypage', '#ZimTikTok #SaTikTok (possibly South Africa TikTok) #viralvideos', "Part 1: Compiled from recommendations given by locals, here are the top 10 dishes you must try in Xinjiang. If you don't taste these, you're missing out! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltips #musteatfood #viral #fyp", 'Nightlife atmosphere in Urumqi, Xinjiang Uighur Autonomous Region, China #urumchi #urumqinightlife #xinjiangnightlife #xinjiang #freeuyghurs #freeugyhurs #freeuyghurs #chinatiktok #china #chinesetiktok #nightlifeinurumchi #urumchi #nightlifeinxinjiang #dunganpeople #uighursinchina #UyghursInChina #freeouighours #uyghurlife #freedom4uyghurs #uighurcauseinchina #urumqi #nightlifeinurumqi #urumqi #Urumqi #foryou #fyp #foryoupage #viral', 'The ending is touching #TikTokVoiceFeature #ChineseMuslim #Muslim #China #HalalFood #XinjiangFood #Culinary #Viral #ViralVideo #ViralTikTok #FYP #FYPChallenge #FYPViral #FYPPage #FYPChallengeViral #FYPSounds', '10 Days in Xinjiang: A Journey Beyond Imagination! From the vibrant Urumqi to the enchanting Kanas, join us for an adventure unlike any other. #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp', 'His nose is really good, dear #dilrabadilmurat #dilraba #dilireba #fyp #fyp #viral', "This is our incredible travel guide for Northern Xinjiang! Reach out to us directly, and we'll assist you in planning everything! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", 'Hidden camps #viral #Islam #Muslim #Uyghur #foryourpage #spreadawareness #help #foryourpage', "Were you aware that Xinjiang is divided into both South and North regions? Watch this video for immediate understanding! Let's discover it together! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltips #viral #fyp", "Here's our 10-day trip plan for Xinjiang! If you like it, let's plan a trip together next year! Feel free to reach out to us, and we can organize the Xinjiang adventure you want! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltip #viral #fyp", 'Replying to @hartantiyunida95 #dewidilraba #dilrabadilmurat #asianbeautytiktok #2023 #queen #goddess #beautiful #xinjiang #uighur #fyp #foryou #video #viral #tiktok', "This is our amazing travel guide for Northern Xinjiang! Get in touch with us directly, and we'll help you plan everything! #xinjiang #travellife #traveltochina #mustvisitplaces #traveltips #trending #foryoupage", 'Spotted Kim Kardashian in Xinjiang! Would you be interested in checking it out? #xinjiang #traveltochina #bigbottoms #traveltiktok #travellife #viralfood #roastedlamb #viralvideo #fyp', 'The atrocities of Xinjiang, part 6. #urgent #emergency #world #worldtiktok #worldwide #foryou #foryoutiktok #tiktok #foryoupage #foryourpage #fypage #fyp #fyi #fypシ #fypツ #fypageシ #video #viralvideo #foryou #viraltiktok #chinatiktok #holocaust #uyghurs #uyghurpeople #uighurs #china #xinjiang #women #8m #feminism #terrortiktok #terror', 'Animals in China freeze due to extremely low temperatures #2024 #viral #foryou #trending #china #xinjiang #newearth #5d #weather #news', "Don't go overseas, because visiting Xinjiang is like experiencing the entire world in one place! #xinjiang #travellife #traveltochina #mustvisitplaces #viralvideo #fyp", 'Took a journey to Xinjiang and uncovered the purpose of life. Our Xinjiang group tour is now available! Come and ask about it now! #xinjiang #travellife #traveltochina #mustvisitplaces #spring #springscentsation #viral #fyp', 'Breathtaking places in Xinjiang. #work #scenery #China #foryoupage #foryou #goingviral #trending #Xinjiang', '#WestKalimantanPost #PostWestKalimantan #fyp #fyp #viral #fypage #viralvideo #viral #tiktokviral #video #development #China', "Can't choose between Southern or Northern Xinjiang? Why not explore both! Take a look at our 14-day plan covering both areas of Xinjiang! #xinjiang #china #traveltips #travellife #traveltochina #mustvisit #vacation #scenery #fyp #viral", 'Children from Xinjiang bring pure joy to the Forbidden City with their captivating dance! #foryoupage #viral #trending #dance #chinese #tiktok', 'The place in the world that is farthest from the sea is in China! Everyone should visit at least once! #xinjiang #travelinglifestyle #traveltochina #mustvisitplaces #trending #foryourpage', 'Part 1: Top 10 Highlights of Xinjiang - A Must-Visit to Reveal the Beauty of Xinjiang #xinjiang #travellife #traveltochina #mustvisitplaces #viral #fyp', 'History repeats itself. #Uyghur #UyghurLivesMatter #SecondWorldWar #Jew #Jews #Jewish #Muslim #War #ConcentrationCamps #HistoryRepeats #America #Muslims #ForYou #Viral #ForYouPage #ViralVideo #Like #Wars', 'As joyful music played, the Tajiks danced warmly to welcome the guests! #xinjiang #Tajik', "Girls from Xinjiang are natural-born dancers! The little girl's elegant movements are in perfect harmony with the music, displaying her unique dance ability. Get ready to be captivated! #Xinjiang #dance #talent #cutegirl", 'Check this out! Can you keep up with them? (Credit to Humar 328 on Douyin) #xinjiang #dance #kids #interesting', 'Check this out! Can you keep up with them? (Credit to Humar 328 on Douyin) #xinjiang #dance #kids #interesting', 'Uyghur girls performed an impromptu dance in the ancient city of Kashgar. A Uyghur girl spontaneously danced to Kazakh music. She danced confidently in front of the camera, demonstrating a unique exotic flair. #xinjiang', "The dance and music traditions of Xinjiang are a lively portrayal of the region's multiculturalism, shaped by various ethnic groups such as the Uygur, Kazak, Kyrgiz, and Tajik. The exceptional fusion of traditional music and dance is a fascinating aspect of Xinjiang's abundant cultural legacy. Thank you for sharing your stunning culture with the world, #Xinjiang! #Uygur #Kazak #Kyrgiz #Tajik #traditionalmusic #dancing #culturalheritage #fyp #Yining.", 'She is incredibly beautiful! Dilraba is performing a traditional Uyghur dance at the Chinese Spring Festival 2024 in Xinjiang. #ForYourPage #Beauty #Girl #Dance #TikTok #Chinese #Spring #Festival', '#beauty #girls #dance #xinjiangchina', '#Xinjiang #XinjiangChina #XinjiangCotton #UygurMuslimsLifeMatter #XinjiangReeducationCamps #XinjiangDance #Uyghur #XinjiangUyghurs #XinjiangA', 'Do I look like an Uygur? #China #Chinese #ChineseLanguage #Mandarin #learn #study #uygur #language #love #dance #dress #bilingual #vicali #hello #thanks #Xinjiang', "If you visit Xinjiang, you can run across the grasslands, dance by the lakeside, sing in the mountains... and do everything you've always wanted to do in this paradise-like location. #Xinjiang #travel #havefun", 'Natural Kashgar dance, China, travel, Xinjiang China, for your page, Uyghur Muslims, Xinjiang Uyghurs, Uyghur, Xinjiang, fitness motivation', 'Grandpa and two beauties are dancing the Xinjiang dance in the ancient city of Kashgar, Xinjiang #Xinjiang #XinjiangChina #XinjiangUyghurs #Uyghur #UyghurMuslims #China', 'Uighur policewomen from Xinjiang were invited to dance together, debunking Western accusations of China conducting a genocide. In China, the 56 ethnic groups are referred to collectively as the Chinese nation, which has remained united for millennia. Meanwhile, the real genocide would be the massacre of Native Indians in the United States. #china #xinjiang #police', 'At a Uyghur wedding ceremony, the bride dances with elegance and happiness, celebrating love in the most captivating way. #Xinjiang #wedding #dance #bride', 'Traditional Kashgar dance from China. Tags: travel, Xinjiang China, for your page, Uyghur Muslims, Xinjiang Uyghurs, Uyghur, Xinjiang, fitness motivation.', 'Children from Xinjiang bring pure joy to the Forbidden City with their captivating dance! #foryourpage #viral #trending #dance #chinese #tiktok', "Participate in the lively festivities as friends from Kashgar in Xinjiang take on the #LoongYearLoongDance challenge. Let's come together in cheerfulness and joyfully dance our way into a prosperous and happy year!", 'The dances of ethnic minorities in Xinjiang are unique!', 'The latest dance trend in China, "#Kemusan," embodies the joyful lifestyle of the Chinese people. This joy is expressed through the elegant movements of Xinjiang performers, captivating everyone in Qatar who sees it! #2024XinjiangCulturalandTourismPromotionWeek #Kemusan #XinjiangMusicandDance #XinjiangIsaNicePlace #XinCulture #ThisIsXinjiang #DreamyXinjiang', "Join us in celebrating the Chinese Lunar New Year with an exhilarating opening dance at the CCTV Spring Festival Gala 2024 in the Kashi Sub-venue! Let's start the Year of the Dragon with infectious enthusiasm and boundless happiness! #HappyChineseLunarNewYear #Xinjiang #springfestival #Gala", "Join us in celebrating the Chinese Lunar New Year with a thrilling opening dance at the 2024 CCTV Spring Festival Gala's Kashi sub-venue! Let's start the Year of the Dragon with contagious energy and unbounded happiness! #HappyChineseLunarNewYear #Xinjiang #springfestival #Gala", '#Bagua City, #Xinjiang #Xinjiang Bagua City', 'Aerial View of Korla City, Xinjiang, Korla', 'Two sides of one city #Shanghai #China', 'The following content tags are related to the question "Where did the Forbidden City get its name?": Forbidden City, Chinese History, Chinese Art, Ancient China, Emperor, Tiananmen Square, Ancient History, Chinese Architecture, Architectural History, Urban History, Urban Planning, History, Chinese, Douyin, Douyin China, China, Chinese Politics, Green Screen.', 'The inhabitants of the ancient city of Kashgar employ their distinctive musical instruments, attire, and their most genuine smiles to present a unique perspective of Xinjiang and Chinese culture to the world. #Xinjiang #ancientcity #music', 'Known as the "capital of the geographical center of the Asian continent," Urumqi, Xinjiang, is the world\'s largest city that is furthest away from the ocean. #Xinjiang #city', 'Understood it in exactly 3 questions... where did you think she was from? #ethnicity #country #india #indian #tibet #tibetan', 'If you really want to immerse yourself in the dynamic culture of Xinjiang, go directly to Kashgar! This old city is like a time capsule, its streets filled with original residents. Explore the hidden corners and alleys, and be prepared to be pleasantly surprised at every turn. #Xinjiang #culture', 'On the 15th, in Turpan, Xinjiang, the hair of tourists stood on end in mid-air. Netizens: Hurry up and leave, the static electricity is too strong, there will be lightning soon, be aware of safety.', '#Bagua City, #Xinjiang #Xinjiang Bagua City', 'Welcome you to travel to the ancient city of Kashgar, Xinjiang. #Uyghur #Uyghur #XinjiangChina #Xinjiang #XinjiangUyghurs #Xinjiang #UyghurMuslims #Travel', 'Grandpa and two beautiful women perform Xinjiang dance in the ancient city of Kashgar, Xinjiang. #Xinjiang #XinjiangChina #XinjiangUyghurs #Uyghur #UyghurMuslims #China', 'Good night, Urumqi. #foryourpage #Xinjiang #Urumqi #urumqi_xinjiang #sunset #goodnight #city', 'On the 15th, in Turpan, Xinjiang, the hair of tourists stood on end. Netizen: Hurry up, the static electricity is too strong, there will be lightning soon, pay attention to safety.', 'Replying to @worldfactsofficial Your video is a fake video. The Chinese city on your list is much cleaner than the US city which is full of homeless people scattered across the streets like trash.', "The Ice and Snow World in Harbin is one of the world's largest winter attractions. This entire ice city is constructed in a month with a cost reaching up to 500 million dollars and it can only stay intact for two months for visitors before it melts away. #iceandsnow #harbin #china #harbinicefestival #icefestival", 'Kids from Xinjiang bring pure joy to the Forbidden City with their captivating dance! #foryourpage #viral #trending #dance #chinese #tiktok', 'Lady from Wulumuqi City, Xinjiang Uygur Autonomous Region, China. #chinesegirl #chinesewoman #chineselady', 'Uyghur girls performed an impromptu dance in the ancient city of Kashgar. A Uyghur girl improvised a dance to Kazakh music. She danced confidently in front of the camera, exhibiting a unique, exotic flair. #xinjiang', 'The only city in China without traffic lights #scenery #xinjiang #bagua #city #incredible #foryou', 'Urumqi, my love! Have you witnessed the magic of the sunrise here? #Urumqi #SunriseMagic #Xinjiang #city #fyp #rotateyourphone', 'Welcome to the ancient city of Kashgar, Xinjiang, China #China #Freestyle #XinjiangChina #Uyghur #Travel #UyghurMuslims #FYP #ReTokForNature #XinjiangUyghurs#', 'Xinjiang is a fantastic place for a road trip. Traverse through the alluring, ancient city of Kashgar, savor the local cuisine, and revel in the snow-capped mountains. Visit Xinjiang to experience the customs of the Western region. #xinjiang #food']
preprocessed_texts = [text.lower() for text in preprocessed_texts]
df = pd.DataFrame(preprocessed_texts, columns=['texts'])

## Extract Features (Embed)

In [ ]:
client = openai.OpenAI(api_key = OPENAI_API_KEY)

def get_embedding(text):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=OPENAI_EMBEDDING_MODEL).data[0].embedding

tqdm.pandas(desc="Embedding")
embeddings = pd.DataFrame(df['texts'].progress_apply(get_embedding).tolist())

Embedding:  44%|███████████████████████████████████████████████████████▌                                                                      | 226/512 [01:42<02:04,  2.30it/s]

## Reduce Dimensionality

In [ ]:
#Initialize UMAP
reducer = umap.UMAP(n_neighbors=15, n_components=2, metric='euclidean')

# Fit and transform the embeddings
umap_embeddings = reducer.fit_transform(embeddings)

# Convert the UMAP results into a DataFrame for easier handling
columns=['UMAP_1', 'UMAP_2']
umap_df = pd.DataFrame(umap_embeddings, columns=columns)

## Cluster

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Range of clusters to try
K_range = range(1, 11)  # Typically from 1 to 10 clusters, adjust as needed

# Sum of squared distances for each k
ssd = []
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(umap_df[columns])
    ssd.append(kmeans.inertia_)  # Inertia: Sum of squared distances of samples to their closest cluster center

# Plotting the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, ssd, 'bo-')  # blue circle markers with solid line
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of squared distances (SSD)')
plt.title('Elbow Method For Optimal k')
plt.grid(True)
plt.show()


In [ ]:
# Initialize KMeans
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0)

# Fit KMeans on the UMAP data
umap_df['Cluster'] = kmeans.fit_predict(umap_df[columns])

# Print the score
score = silhouette_score(umap_df[columns], kmeans.labels_)
print(f'silhouette score: {score}')

## Visualize

In [ ]:
# Set up the matplotlib figure
plt.figure(figsize=(10, 8))

# Plotting
sns.scatterplot(
    x='UMAP_1', y='UMAP_2',
    hue='Cluster',  # Color by cluster
    palette=sns.color_palette("hsv", n_colors=len(umap_df['Cluster'].unique())),  # Use a color palette that can scale with number of clusters
    data=umap_df,
    legend="full",
    alpha=0.6  # Transparency of the dots
)

plt.title('K-Means Clustering on UMAP-reduced TikTok Embeddings')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.legend(title='Cluster')
plt.grid(True)  # Adds a grid for easier visualization

# Show the plot
plt.show()

## Analyze

In [ ]:
# Concatenate the tiktoks with the cluster labels
tiktoks_with_clusters = pd.concat([df, umap_df['Cluster']], axis=1)

# Display the first few rows to confirm
print(tiktoks_with_clusters.head())

In [ ]:
# Iterate through each cluster and print all the texts
for cluster in sorted(tiktoks_with_clusters['Cluster'].unique()):
    print(f"TikToks in Cluster {cluster}:")
    cluster_texts = tiktoks_with_clusters[tiktoks_with_clusters['Cluster'] == cluster]['texts']
    for text in cluster_texts:
        print(text)
    print("\n")  # Adds a new line for better separation between cluster outputs